## 142 SAT [Enhancement] Märk ut saknade taggar för framkomlighetsbedömning

### version 3.3 

* this [Notebook](https://github.com/salgo60/Stockholm_Archipelago_Trail/blob/main/notebook/142_3%20SAT%20%5BEnhancement%5D%20M%C3%A4rk%20ut%20saknade%20taggar%20f%C3%B6r%20framkomlighetsbed%C3%B6mning.ipynb)
  * Latest maps [Image Quality Control](https://raw.githack.com/salgo60/Stockholm_Archipelago_Trail/main/notebook/output/SAT_ALL_IN_ONE_142_3_image_qc_latest.html) / [Global Audit](https://raw.githack.com/salgo60/Stockholm_Archipelago_Trail/main/notebook/output/SAT_ALL_IN_ONE_142_3_ALL_latest.html) / [Task Map](https://raw.githack.com/salgo60/Stockholm_Archipelago_Trail/main/notebook/output/SAT_ALL_IN_ONE_142_3_split_todo_latest.html) / [Steps](https://raw.githack.com/salgo60/Stockholm_Archipelago_Trail/main/notebook/output/SAT_ALL_IN_ONE_142_3_steps_only_latest.html) / [Water proximity](https://raw.githack.com/salgo60/Stockholm_Archipelago_Trail/main/notebook/output/SAT_ALL_IN_ONE_142_3_water_proximity_latest.html)
 

---- 
* Issue [142 SAT Enhancement Märk ut saknade taggar för framkomlighetsbedömning](https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/142)
 

* Try to get better quality of trail data in OSM

---- 
* Version 3.3 creates three maps

       
    
  
  


In [1]:
import time

from datetime import datetime

now = datetime.now()
timestamp = now.timestamp()

start_time = time.time()
print("Start:", datetime.now().strftime("%Y-%m-%d %H:%M:%S"))

Start: 2025-09-27 08:30:29


In [ ]:
# =================== SAT: ALL-IN-ONE (WD + OSM + AUDIT + POPUP + MAP SAVE) ===================
from __future__ import annotations

# --- Stdlib
import os
import re
import time
import json
import shutil
from pathlib import Path
from urllib.parse import quote, unquote
from string import Template
from datetime import datetime
import html as _html
import html
import shapely 

# --- Third-party
import requests
import pandas as pd
import geopandas as gpd
from shapely.geometry import LineString
import folium
from folium import GeoJson, GeoJsonTooltip, GeoJsonPopup

# =================== Projekt/Output =========================================
PROJECT_NAME = "SAT_ALL_IN_ONE_142_3"
OUTPUT_DIR   = "./output"  # ändra vid behov
stamp        = pd.Timestamp.now().strftime("%Y%m%d_%H%M")

WD_ENDPOINT  = "https://query.wikidata.org/sparql"
OVERPASS_URL = "https://overpass-api.de/api/interpreter"
UA           = {"User-Agent": "SAT-etapper/1.0 (salgo60 tools)"}

WIKI_KEY_URLS = {
    "trail_visibility": "https://wiki.openstreetmap.org/wiki/Key:trail_visibility",
    "sac_scale":        "https://wiki.openstreetmap.org/wiki/Key:sac_scale",
    "mtb:scale":        "https://wiki.openstreetmap.org/wiki/Key:mtb:scale",
    "mtb:scale:imba":   "https://wiki.openstreetmap.org/wiki/Key:mtb:scale:imba",
    "surface":          "https://wiki.openstreetmap.org/wiki/Key:surface",
    "smoothness":       "https://wiki.openstreetmap.org/wiki/Key:smoothness",
    "tracktype":        "https://wiki.openstreetmap.org/wiki/Key:tracktype",
    "highway":          "https://wiki.openstreetmap.org/wiki/Key:highway",
    "foot":             "https://wiki.openstreetmap.org/wiki/Key:foot",
    "bicycle":          "https://wiki.openstreetmap.org/wiki/Key:bicycle",
    "width":            "https://wiki.openstreetmap.org/wiki/Key:width",
    "incline":          "https://wiki.openstreetmap.org/wiki/Key:incline",
    "lit":              "https://wiki.openstreetmap.org/wiki/Key:lit",
    "step_count":       "https://wiki.openstreetmap.org/wiki/Key:step_count",
    "ford":             "https://wiki.openstreetmap.org/wiki/Key:ford",
    "bridge":           "https://wiki.openstreetmap.org/wiki/Key:bridge",
    "tunnel":           "https://wiki.openstreetmap.org/wiki/Key:tunnel",
    "wheelchair":       "https://wiki.openstreetmap.org/wiki/Key:wheelchair",
    "ramp":             "https://wiki.openstreetmap.org/wiki/Key:ramp",
    "osmc:symbol":      "https://wiki.openstreetmap.org/wiki/Key:osmc:symbol",
}

# =================== Konstanter/Färger ======================================
AUDIT_COLORS = {
    "missing_surface":   "#ff8800",
    "missing_access":    "#cc0000",
    "missing_hike":      "#7e57c2",
    "inconsistent":      "#ffd54f",
    "missing_stepcount": "#8d6e63",
    "tracks_compacted":  "#2b8cbe",
    "missing_mtbscale":  "#00acc1",
    "missing_tracktype": "#ef6c00",
    "scenic":            "#43a047",
}

# =================== Hjälpare: OSM-länkar ===================================
def _mk_osm_links(osmid, *, include_visual: bool = True) -> str:
    """Bygger klicklänkar till OSM-view, iD-editor och historiksidor (+ Visual history)."""
    try:
        osmid = int(osmid)
        view   = f"https://www.openstreetmap.org/way/{osmid}"
        edit   = f"https://www.openstreetmap.org/edit?editor=id&way={osmid}"
        hist   = f"https://pewu.github.io/osm-history/#/way/{osmid}"
        hist2  = f"https://osm.mapki.com/history/way/{osmid}"
        vhist  = f"https://aleung.github.io/osm-visual-history/#/way/{osmid}"

        parts = [
            f'<a href="{view}" target="_blank">OSM way {osmid}</a>',
            f'<a href="{edit}" target="_blank">Öppna i iD</a>',
            f'<a href="{hist}" target="_blank">Historik</a>',
            f'<a href="{hist2}" target="_blank">metadata historik</a>',
        ]
        if include_visual:
            parts.append(f'<a href="{vhist}" target="_blank">Visual history</a>')
        return " · ".join(parts)
    except Exception:
        return ""


def _mk_mapillary_link(row) -> str:
    """HTML-länk till Mapillary-foto om tagg finns, annars karta på centroid."""
    mkey = None
    try:
        mkey = row.get("mapillary")
    except Exception:
        pass

    if isinstance(mkey, str) and mkey.strip():
        key = mkey.split(";")[0].strip()
        url = f"https://www.mapillary.com/app/?pKey={quote(key)}&focus=photo"
        return f'<a href="{url}" target="_blank">Mapillary foto</a>'

    try:
        geom = row.get("geometry")
        if geom is not None:
            c = geom.centroid
            lat, lon = float(c.y), float(c.x)
            url = f"https://www.mapillary.com/app/?z=18&lat={lat:.6f}&lng={lon:.6f}&focus=map"
            return f'<a href="{url}" target="_blank">Mapillary här</a>'
    except Exception:
        pass
    return ""
def _format_wikimap_coords(lat: float, lon: float, prec: int = 6) -> tuple[str, str]:
    """
    Wikimap likes zero-padded degrees like:
      lat=059.412525  (3 digits)
      lon=0018.556753 (4 digits)
    Keep sign if S/W, pad absolute value, 6 decimals by default.
    """
    def fmt(v: float, width: int) -> str:
        sign = "-" if v < 0 else ""
        return f"{sign}{abs(v):0{width}.{prec}f}"
    return fmt(lat, 3), fmt(lon, 4)

def _mk_wikimap_link(row, zoom: int = 16, cluster: bool = False, wp: bool = False) -> str:
    """HTML-link to Wikimap at the feature centroid with padded coords."""
    try:
        geom = row.get("geometry")
        if geom is None:
            return ""
        c = geom.centroid
        lat, lon = float(c.y), float(c.x)
        s_lat, s_lon = _format_wikimap_coords(lat, lon, prec=6)
        url = (
            "https://wikimap.toolforge.org/"
            f"?wp={'true' if wp else 'false'}"
            f"&cluster={'true' if cluster else 'false'}"
            f"&zoom={int(zoom)}&lat={s_lat}&lon={s_lon}"
        )
        return f'<a href="{url}" target="_blank">Wikimap här</a>'
    except Exception:
        return "" 
def _mk_all_links(row_or_osmid) -> str:
    osmid = row_or_osmid.get("osmid") if hasattr(row_or_osmid, "get") else row_or_osmid
    return " · ".join(filter(None, [
        _mk_osm_links(osmid, include_visual=True),
        _mk_wikimap_link(row_or_osmid if hasattr(row_or_osmid, "get") else {"geometry": None}),
        _mk_mapillary_link(row_or_osmid if hasattr(row_or_osmid, "get") else {"geometry": None}),
    ]))
# =================== Steps-only karta =======================================
def make_steps_only_map(etapp_gdfs: dict[str, gpd.GeoDataFrame],
                        OUTPUT_DIR: str = OUTPUT_DIR,
                        PROJECT_NAME: str = PROJECT_NAME,
                        stamp: str = stamp) -> str:
    # 1) Samla allt i en DF
    dfs = []
    for etapp_label, gdf in etapp_gdfs.items():
        if gdf is None or gdf.empty:
            continue
        gf = gdf.copy()
        gf["name"] = gf.get("name", pd.Series([None] * len(gf))).fillna(etapp_label)
        dfs.append(gf)

    if not dfs:
        print("Inga segment att visa (steps).")
        return ""

    df_all = pd.concat(dfs, ignore_index=True).copy()

    # säkerställ kolumner vi använder
    required = ["osmid","geometry","name","highway","surface","tracktype",
                "foot","bicycle","sac_scale","trail_visibility","smoothness","width",
                "mtb:scale","mtb:scale:imba","osmc:symbol","image","incline","lit",
                "step_count","ford","bridge","tunnel","wheelchair","ramp","source","mapillary"]
    for c in required:
        if c not in df_all.columns:
            df_all[c] = None

    # 2) Filtrera ut enbart trappor
    H = df_all["highway"].astype(str).str.lower().replace({"none": ""})
    steps = df_all.loc[H == "steps"].copy()
    if steps.empty:
        print("Hittade inga trappor (highway=steps).")
        return ""

    # 3) Popup-byggare (återanvänder existerande badges, länkar m.m.)
    def _build_popup_html_from_row(row):
        tags = {k: row.get(k) for k in [
            "highway","foot","bicycle","sac_scale","trail_visibility","surface","tracktype","smoothness","width",
            "mtb:scale","mtb:scale:imba","osmc:symbol","image","incline","lit","step_count","ford","bridge",
            "tunnel","wheelchair","ramp","source","mapillary"
        ]}
        title = row.get("name") or f"Way {row.get('osmid')}"
        links = " · ".join(filter(None, [
            _mk_osm_links(row.get("osmid")),
            _mk_wikimap_link(row),
            _mk_mapillary_link(row),
        ]))
        extra = f'<div style="margin-top:8px;">{links}</div>'
        return build_hiking_popup_html(tags, title=title, osmid=row.get("osmid"), extra_html=extra)

    steps["popup_html"] = steps.apply(_build_popup_html_from_row, axis=1)

    # 4) Masker: med/utan step_count
    s = steps["step_count"].fillna("").astype(str).str.strip().str.lower()
    has_count = (s != "") & (s != "nan")
    no_count  = ~has_count

    # 5) Karta + lager
    gdf_steps = gpd.GeoDataFrame(steps, geometry="geometry", crs="EPSG:4326")
    minx, miny, maxx, maxy = gdf_steps.total_bounds
    m = folium.Map(location=[(miny + maxy) / 2, (minx + maxx) / 2],
                   zoom_start=12, control_scale=True, tiles="OpenStreetMap")

    # ljus bakgrund av alla etapp-geometrier (valfritt)
    try:
        bg_all = pd.concat([gdf.loc[:, ["geometry"]] for gdf in etapp_gdfs.values() if gdf is not None and not gdf.empty],
                           ignore_index=True)
        if not bg_all.empty:
            GeoJson(bg_all, name="Etapper (bakgrund)",
                    style_function=lambda _f: {"weight": 3, "color": "#666", "opacity": 0.35}).add_to(m)
    except Exception:
        pass

    def _add_steps_layer(gdf_sel: gpd.GeoDataFrame, name: str, color_hex: str):
        if gdf_sel.empty:
            return
        gj = GeoJson(
            data=gdf_sel[["geometry", "popup_html", "name", "step_count"]],
            name=name,
            style_function=lambda _f, _c=color_hex: {"weight": 6, "color": _c},
            tooltip=GeoJsonTooltip(fields=["name", "step_count"],
                                   aliases=["Section", "step_count"],
                                   sticky=True),
            popup=GeoJsonPopup(fields=["popup_html"], aliases=[""], parse_html=True, labels=False, max_width=520)
        )
        fg = folium.FeatureGroup(name=name)
        gj.add_to(fg)
        fg.add_to(m)

    COLOR_HAS_COUNT = "#1f6feb"                # blå
    COLOR_NO_COUNT  = AUDIT_COLORS["missing_stepcount"]  # brun från din audit-palett

    _add_steps_layer(gdf_steps.loc[has_count].copy(), "Trappor ✔ med step_count", COLOR_HAS_COUNT)
    _add_steps_layer(gdf_steps.loc[no_count].copy(),  "Trappor ⚠ saknar step_count", COLOR_NO_COUNT)

    # enkel legend
    legend_html = f"""
    <div style="position: fixed; bottom: 18px; left: 18px; z-index: 9999;
                background: rgba(255,255,255,0.92); padding: 10px 12px; border-radius: 10px;
                box-shadow: 0 2px 8px rgba(0,0,0,0.15); font-size: 13px;">
      <div style="font-weight:600; margin-bottom:6px;">Trappor (highway=steps)</div>
      <div><span style="display:inline-block;width:14px;height:3px;background:{COLOR_HAS_COUNT};margin-right:6px;"></span>✔ med step_count</div>
      <div><span style="display:inline-block;width:14px;height:3px;background:{COLOR_NO_COUNT};margin-right:6px;"></span>⚠ saknar step_count</div>
    </div>
    """
    m.get_root().html.add_child(folium.Element(legend_html))
    folium.LayerControl(collapsed=False).add_to(m)

    # fit bounds & spara
    m.fit_bounds([[gdf_steps.total_bounds[1], gdf_steps.total_bounds[0]],
                  [gdf_steps.total_bounds[3], gdf_steps.total_bounds[2]]])

    os.makedirs(OUTPUT_DIR, exist_ok=True)
    html_path = os.path.join(OUTPUT_DIR, f"{PROJECT_NAME}_steps_only_{stamp}.html")
    add_about_box(m, issue_number=142, map_name="Steps only")
    save_with_latest(m, html_path)
    print("Steps-only karta sparad till:", html_path)
    return html_path
    
def summarize_sat(etapp_gdfs: dict[str, gpd.GeoDataFrame]) -> pd.DataFrame:
    rows = []
    for section, gdf in etapp_gdfs.items():
        if gdf is None or gdf.empty:
            continue
        # ensure length in meters
        gdf_m = gdf.to_crs(epsg=3857)  # web mercator meters
        gdf_m["length_m"] = gdf_m.geometry.length

        # summarize per surface
        surf_stats = gdf_m.groupby(gdf_m["surface"].fillna("unknown"))["length_m"].sum().to_dict()
        hwy_stats  = gdf_m.groupby(gdf_m["highway"].fillna("unknown"))["length_m"].sum().to_dict()

        rows.append({
            "section": section,
            "total_km": gdf_m["length_m"].sum() / 1000,
            "surface_stats": surf_stats,
            "highway_stats": hwy_stats,
            "num_steps": int((gdf_m["highway"].str.lower() == "steps").sum()),
            "num_bridges": int((gdf_m["bridge"].str.lower() == "yes").sum()),
            "num_fords": int((gdf_m["ford"].str.lower() == "yes").sum()),
            "with_sac": int(gdf_m["sac_scale"].notna().sum()),
            "with_trail_visibility": int(gdf_m["trail_visibility"].notna().sum()),
        })
    return pd.DataFrame(rows)

# =================== Hjälpare: Commons-bilder ===============================
def _is_http_url(s: str) -> bool:
    return isinstance(s, str) and s.lower().startswith(("http://", "https://"))

def normalize_commons_title(image_tag: str) -> str | None:
    """Normaliserar in till 'File:Title.ext' (stöder /wiki/File:... och upload-URL:er)."""
    if not isinstance(image_tag, str) or not image_tag.strip():
        return None
    s = image_tag.strip()

    m = re.search(r'(?:^|/wiki/)(File:[^#?]+)$', s, flags=re.I)
    if m:
        raw = m.group(1)
        return "File:" + unquote(raw.split(":", 1)[1])

    if "upload.wikimedia.org" in s:
        last = s.split("/")[-1]
        last = re.sub(r'^\d+px-', '', last)   # ta bort tumnagel-prefix
        return "File:" + unquote(last)

    if not _is_http_url(s):
        if s.lower().startswith("file:"):
            return "File:" + unquote(s.split(":", 1)[1])
        return "File:" + unquote(s)
    return None

def commons_thumb_url(title: str, width: int = 600) -> str:
    if not isinstance(title, str) or not title:
        return ""
    if not title.lower().startswith("file:"):
        title = "File:" + title
    decoded = "File:" + unquote(title.split(":", 1)[1])
    safe = quote(decoded, safe="/:")
    return f"https://commons.wikimedia.org/wiki/Special:FilePath/{safe}?width={width}"

def commons_filepage_url(title: str) -> str:
    if not isinstance(title, str) or not title:
        return ""
    if not title.lower().startswith("file:"):
        title = "File:" + title
    decoded = unquote(title.split(":", 1)[1]).replace(" ", "_")
    safe = quote("File:" + decoded, safe="/:")
    return f"https://commons.wikimedia.org/wiki/{safe}"

def get_commons_entity_json_for_title(title: str, timeout=10) -> dict | None:
    if not isinstance(title, str) or not title:
        return None
    if not title.lower().startswith("file:"):
        title = "File:" + title
    decoded = "File:" + unquote(title.split(":", 1)[1])
    url = f"https://commons.wikimedia.org/wiki/Special:EntityData/{quote(decoded, safe='/:')}.json"
    try:
        r = requests.get(url, headers=UA, timeout=timeout)
        if r.status_code == 200:
            return r.json()
    except Exception:
        pass
    return None

def commons_has_p10689(title: str, osmid: int | str) -> tuple[bool, str | None]:
    data = get_commons_entity_json_for_title(title, timeout=6)
    if not data or "entities" not in data:
        return (False, None)
    for mid, ent in data["entities"].items():
        for claim in ent.get("statements", {}).get("P10689", []) or []:
            val = claim.get("mainsnak", {}).get("datavalue", {}).get("value")
            if str(val) == str(osmid):
                return (True, mid)
    return (False, None)

def make_image_html(image_value: str) -> str:
    if not image_value:
        return ""
    title = normalize_commons_title(image_value)
    if title:
        thumb = commons_thumb_url(title, width=600)
        link  = commons_filepage_url(title)
        return (f'<a href="{html.escape(link)}" target="_blank">'
                f'<img src="{html.escape(thumb)}" style="max-width:100%;height:auto;border-radius:8px"/></a>')
    if _is_http_url(image_value):
        return f'<img src="{html.escape(image_value)}" style="max-width:100%;height:auto;border-radius:8px"/>'
    return ""
def first_thumb_url(image_value: str | None, width: int = 320) -> str:
    """
    Returnerar en tumnagel-URL för Commons 'File:*' eller original-URL om det är http(s).
    Inget Mapillary-fallback. Tom sträng om inget går att använda.
    """
    if not image_value:
        return ""
    title = normalize_commons_title(image_value)
    if title:
        return commons_thumb_url(title, width=width)
    if _is_http_url(str(image_value)):
        return str(image_value)
    return ""

# =================== Badge/Popup-hjälpare ===================================
def _fmt(v: str | None) -> str:
    return "" if v is None else str(v).replace("_", " ").replace("-", " ").strip()

def _tag(tags: dict, *keys, default=None):
    for k in keys:
        if k in tags and str(tags[k]).strip():
            return tags[k]
    return default

def _badge(text: str, kind: str = "default") -> str:
    palette = {
        "primary":"#1f6feb","success":"#238636","warning":"#9e6a03",
        "danger":"#b62324","muted":"#57606a","default":"#6e7781",
    }
    color = palette.get(kind, palette["default"])
    return (f'<span style="display:inline-block;padding:2px 8px;border-radius:999px;'
            f'font-size:12px;line-height:18px;background:{color};color:white;'
            f'margin:0 6px 6px 0;white-space:nowrap;">{html.escape(text)}</span>')

def _sac_badge(val: str | None) -> str:
    if not val: return ""
    v = str(val).lower()
    name = {
        "hiking":"hiking",
        "mountain_hiking":"mountain hiking",
        "demanding_mountain_hiking":"demanding mountain hiking",
        "alpine_hiking":"alpine hiking",
        "demanding_alpine_hiking":"demanding alpine hiking",
        "difficult_alpine_hiking":"difficult alpine hiking",
    }.get(v, v)
    kind = "success" if v=="hiking" else ("warning" if v in {"mountain_hiking","demanding_mountain_hiking"} else "danger")
    return _badge(f"sac_scale: {_fmt(name)}", kind)

def _trail_visibility_badge(val: str | None) -> str:
    if not val: return ""
    v = str(val).lower()
    kind = "success" if v in {"excellent","good"} else ("warning" if v=="intermediate" else ("danger" if v in {"bad","none"} else "default"))
    return _badge(f"visibility: {_fmt(v)}", kind)

def _surface_badge(val: str | None) -> str:     return _badge(f"surface: {_fmt(val)}","muted") if val else ""
def _width_badge(val: str | None) -> str:       return _badge(f"width: {_fmt(val)}","muted")   if val else ""
def _smoothness_badge(val: str | None) -> str:  return _badge(f"smoothness: {_fmt(val)}","muted") if val else ""
def _osmc_symbol_badge(val: str | None) -> str: return _badge(f"osmc:symbol: {_fmt(val)}","primary") if val else ""
def _sat_color_hint(val: str | None) -> str:
    if not val: return ""
    return _badge("SAT blå/gul","primary") if ("blue" in val.lower() and "yellow" in val.lower()) else ""
def _foot_badge(val: str | None) -> str:
    if not val: return ""
    v = str(val).lower()
    return _badge(f"foot: {_fmt(v)}", "success" if v in {"designated","yes"} else "default")
def _highway_badge(val: str | None) -> str:     return _badge(f"highway: {_fmt(val)}","default") if val else ""
def _incline_badge(val: str | None) -> str:     return _badge(f"incline: {val}","muted") if val else ""
def _lit_badge(val: str | None) -> str:
    if not val: return ""
    v = str(val).lower()
    return _badge("lit","success") if v=="yes" else (_badge("unlit","muted") if v=="no" else _badge(f"lit: {v}","muted"))
def _steps_badge(step_count: str | None, highway: str | None) -> str:
    return _badge(f"steps: {step_count}","default") if (str(highway).lower()=="steps" and step_count) else ""
def _crossings_badge(ford: str | None, bridge: str | None, tunnel: str | None) -> str:
    chips = []
    if str(bridge).lower()=="yes": chips.append(_badge("bridge","primary"))
    if str(ford).lower()=="yes":   chips.append(_badge("ford","warning"))
    if str(tunnel).lower()=="yes": chips.append(_badge("tunnel","default"))
    return "".join(chips)
def _wheelchair_badge(val: str | None) -> str:
    if not val: return ""
    v = str(val).lower()
    return _badge(f"wheelchair: {v}", "success" if v=="yes" else ("warning" if v=="limited" else "muted"))
def _source_badge(val: str | None) -> str:      return _badge(f"source: {_fmt(val)}", "primary") if val else ""

def _collect_images_from_tags(tags: dict, max_n: int = 6) -> list[str]:
    imgs = []
    base = _tag(tags, "image")
    if base: imgs.append(base)
    for i in range(max_n):
        k = f"image:{i}"
        if k in tags and str(tags[k]).strip():
            imgs.append(tags[k])
    seen, uniq = set(), []
    for it in imgs:
        if it not in seen:
            uniq.append(it); seen.add(it)
    return uniq[:max_n]

def _render_images(img_values: list[str]) -> str:
    if not img_values: return ""
    cards = []
    for v in img_values:
        img_html = make_image_html(v)
        if img_html:
            cards.append(f'<div style="flex:1 1 240px;max-width:100%;padding:4px;">{img_html}</div>')
    if not cards: return ""
    return ('<div style="display:flex;flex-wrap:wrap;gap:4px;margin-top:8px;'
            'align-items:flex-start;justify-content:flex-start;">' + "".join(cards) + "</div>")

def _collect_source_subtags(tags: dict) -> list[str]:
    return [f"{k}={tags[k]}" for k in sorted(tags) if k.startswith("source:") and str(tags[k]).strip()]

# =================== Popup: Vandringsfokus ==================================
def build_hiking_popup_html(tags: dict, title: str | None = None, osmid=None, extra_html: str = "") -> str:
    highway    = _tag(tags, "highway")
    foot       = _tag(tags, "foot")
    sac        = _tag(tags, "sac_scale")
    visibility = _tag(tags, "trail_visibility")
    surface    = _tag(tags, "surface")
    tracktype  = _tag(tags, "tracktype")
    width      = _tag(tags, "width")
    smooth     = _tag(tags, "smoothness")
    mtb_scale  = _tag(tags, "mtb:scale", "mtb:scale:imba")
    bicycle    = _tag(tags, "bicycle")
    osmc_sym   = _tag(tags, "osmc:symbol")

    incline    = _tag(tags, "incline")
    lit        = _tag(tags, "lit")
    step_count = _tag(tags, "step_count")
    ford       = _tag(tags, "ford")
    bridge     = _tag(tags, "bridge")
    tunnel     = _tag(tags, "tunnel")
    wheelchair = _tag(tags, "wheelchair")
    ramp       = _tag(tags, "ramp")
    source_tag = _tag(tags, "source")

    badges_primary = "".join([_highway_badge(highway), _foot_badge(foot), _sac_badge(sac)])
    badges_nav     = _trail_visibility_badge(visibility)
    badges_surface = "".join(filter(None, [
        _surface_badge(surface),
        _badge(f"tracktype: {_fmt(tracktype)}","muted") if tracktype else ""
    ]))
    badges_cycling = "".join(filter(None, [
        _width_badge(width), _smoothness_badge(smooth),
        _badge(f"mtb:scale: {_fmt(mtb_scale)}","default") if mtb_scale else "",
        _badge(f"bicycle: {_fmt(bicycle)}","default") if bicycle else "",
        _osmc_symbol_badge(osmc_sym), _sat_color_hint(osmc_sym),
    ]))
    badges_cross = "".join(filter(None, [
        _crossings_badge(ford, bridge, tunnel),
        _steps_badge(step_count, highway),
        _incline_badge(incline),
        _lit_badge(lit),
        _wheelchair_badge(wheelchair),
    ]))

    src_sub       = _collect_source_subtags(tags)
    badges_source = " ".join(_badge(s, "primary") for s in src_sub) or _source_badge(source_tag)

    warnings = []
    if (highway in {"path","footway"}) and (str(foot).lower() in {"designated","yes"}):
        miss = []
        if not sac:        miss.append("sac_scale")
        if not visibility: miss.append("trail_visibility")
        if miss:
            warnings.append(_badge("⚠ saknar " + ", ".join(miss), "danger"))
    if tracktype in {"grade1","grade2","grade3","grade4","grade5"} and highway != "track":
        warnings.append(_badge("⚠ tracktype utan highway=track", "warning"))

    img_values = _collect_images_from_tags(tags)
    img_html   = _render_images(img_values)

    if osmid is not None and img_values:
        t = normalize_commons_title(img_values[0])
        if t:
            ok, _ = commons_has_p10689(t, osmid)
            if ok:
                warnings.append(_badge("✔ Bild kopplad via P10689", "success"))

    title_html = f'<div style="font-weight:700;font-size:18px;margin:0 0 6px 0;">{html.escape(title)}</div>' if title else ""
    desc_bits  = []
    if sac:        desc_bits.append(f"sac_scale: {_fmt(sac)}")
    if visibility: desc_bits.append(f"visibility: {_fmt(visibility)}")
    if surface:    desc_bits.append(f"surface: {_fmt(surface)}")
    desc_html = ('<div style="color:#57606a;font-size:13px;margin:2px 0 8px 0;">'
                 "Vandringsprofil: " + ", ".join(map(html.escape, desc_bits)) + "</div>") if desc_bits else ""
    warning_html = "".join(warnings)

    def section(emoji, text, content):
        if not content: return ""
        return (f'<div style="margin:6px 0 2px 0;font-weight:600;font-size:13px;">{emoji} {text}</div>'
                f'<div style="margin:4px 0 8px 0;">{content}</div>')

    present_keys = []
    for k in ["highway","foot","bicycle","sac_scale","trail_visibility","surface","tracktype",
              "smoothness","width","mtb:scale","mtb:scale:imba","osmc:symbol",
              "incline","lit","step_count","ford","bridge","tunnel","wheelchair","ramp"]:
        v = _tag(tags, k)
        if v is not None and str(v).strip():
            present_keys.append(k)

    doc_links = []
    for k in present_keys:
        wl = _wiki_link_for_key(k)
        if wl: doc_links.append(wl)
        doc_links.append(_taginfo_link_for_key(k))

    html_sections = "".join([
        section("🥾","Grundtaggar", badges_primary),
        section("🧭","Synlighet & orientering", badges_nav),
        section("🪵","Underlag", badges_surface),
        section("🚲","Cykel / MTB", badges_cycling),
        section("🌉","Passager & hinder", badges_cross),
        section("🗂️","Källa", badges_source),
        section("📚","Dokumentation", "".join(doc_links)),
        section("🖼️","Bilder", img_html),
    ])

    outer = (
        '<div style="max-width:520px">'
        '<div style="border-radius:12px;padding:10px;background:#ffffff;'
        'box-shadow:0 1px 6px rgba(0,0,0,.08);font-family:system-ui,-apple-system,'
        'Segoe UI,Roboto,Ubuntu,Helvetica,Arial,sans-serif;font-size:14px;color:#24292f;">'
        f'{title_html}'
        '<div style="display:flex;align-items:center;gap:8px;margin:4px 0 8px 0;">'
        '<div style="font-weight:700;">🥾 Vandring</div>'
        f'{warning_html}'
        '</div>'
        f'{desc_html}{html_sections}{extra_html}'
        "</div></div>"
    )
    return outer

# =================== Legend ==================================================
def add_audit_legend(m: folium.Map, include_tracks=False):
    lines = [
      f'<div><span style="display:inline-block;width:14px;height:3px;background:{AUDIT_COLORS["missing_surface"]};margin-right:6px;"></span>saknar <code>surface=*</code></div>',
      f'<div><span style="display:inline-block;width:14px;height:3px;background:{AUDIT_COLORS["missing_access"]};margin-right:6px;"></span>saknar <code>foot=*</code> / <code>bicycle=*</code></div>',
      f'<div><span style="display:inline-block;width:14px;height:3px;background:{AUDIT_COLORS["missing_hike"]};margin-right:6px;"></span>saknar <code>sac_scale</code> / <code>trail_visibility</code></div>',
      f'<div><span style="display:inline-block;width:14px;height:3px;background:{AUDIT_COLORS["inconsistent"]};margin-right:6px;"></span><code>tracktype=*</code> utan <code>highway=track</code></div>',
      f'<div><span style="display:inline-block;width:14px;height:3px;background:{AUDIT_COLORS["missing_stepcount"]};margin-right:6px;"></span><code>highway=steps</code> utan <code>step_count=*</code></div>',
      f'<div><span style="display:inline-block;width:14px;height:3px;background:{AUDIT_COLORS["missing_mtbscale"]};margin-right:6px;"></span>saknar <code>mtb:scale</code> / <code>mtb:scale:imba</code></div>',
      f'<div><span style="display:inline-block;width:14px;height:3px;background:{AUDIT_COLORS["missing_tracktype"]};margin-right:6px;"></span><code>highway=track</code> utan <code>tracktype=*</code></div>',
      f'<div><span style="display:inline-block;width:14px;height:3px;background:{AUDIT_COLORS["scenic"]};margin-right:6px;"></span><code>scenic=yes</code> (tematiskt lager)</div>',
    ]
    if include_tracks:
        lines.append(f'<div><span style="display:inline-block;width:14px;height:3px;background:{AUDIT_COLORS["tracks_compacted"]};margin-right:6px;"></span>Tracks (<code>surface=compacted</code>)</div>')
    html_legend = f"""
    <div style="position: fixed; bottom: 18px; left: 18px; z-index: 9999;
                background: rgba(255,255,255,0.92); padding: 10px 12px; border-radius: 10px;
                box-shadow: 0 2px 8px rgba(0,0,0,0.15); font-size: 13px;">
      <div style="font-weight:600; margin-bottom:6px;">Datakvalitet (globala lager)</div>
      {''.join(lines)}
    </div>
    """
    m.get_root().html.add_child(folium.Element(html_legend))

# =================== Wikidata: hämta SAT-etapper ============================
def fetch_sat_sections_from_wikidata(trail_qid: str = "Q131318799") -> pd.DataFrame:
    query = f"""
    SELECT ?section ?sectionLabel ?osmr ?osmw_p10689 ?osmw_p11693 WHERE {{
      VALUES ?trail {{ wd:{trail_qid} }}
      {{ ?section wdt:P361+ ?trail . }} UNION {{ ?trail wdt:P527 ?section . }}
      FILTER(?section != ?trail)
      OPTIONAL {{ ?section wdt:P402 ?osmr }}
      OPTIONAL {{ ?section wdt:P10689 ?osmw_p10689 }}
      OPTIONAL {{ ?section wdt:P11693 ?osmw_p11693 }}
      SERVICE wikibase:label {{ bd:serviceParam wikibase:language "sv,en". }}
    }}
    ORDER BY ?sectionLabel
    """
    headers = {"Accept": "application/sparql-results+json", **UA}
    r = requests.get(WD_ENDPOINT, params={"query": query}, headers=headers, timeout=60)
    r.raise_for_status()
    data = r.json()
    rows = []
    for b in data["results"]["bindings"]:
        qid   = b["section"]["value"].split("/")[-1]
        label = b.get("sectionLabel", {}).get("value", qid)
        osmr  = b.get("osmr", {}).get("value")
        osmw1 = b.get("osmw_p10689", {}).get("value")
        osmw2 = b.get("osmw_p11693", {}).get("value")
        osmw  = osmw1 or osmw2
        rows.append({
            "section_qid": qid,
            "sectionLabel": label,
            "osmr": pd.to_numeric(osmr, errors="coerce"),
            "osmw": pd.to_numeric(osmw, errors="coerce"),
        })
    df = pd.DataFrame(rows)
    if not df.empty:
        df["osmr"] = df["osmr"].astype("Int64")
        df["osmw"] = df["osmw"].astype("Int64")
    return df

# =================== Overpass-hjälpare ======================================
def _overpass(query: str, retries: int = 3, pause: float = 2.0) -> dict:
    for i in range(retries):
        try:
            resp = requests.post(OVERPASS_URL, data={"data": query}, headers=UA, timeout=180)
            if resp.status_code == 429 and i < retries-1:
                time.sleep(pause * (i+1)); continue
            resp.raise_for_status()
            return resp.json()
        except Exception:
            if i == retries-1:
                raise
            time.sleep(pause * (i+1))
    return {"elements": []}
# =================== POI (toaletter & dricksvatten) =========================
POI_KEYS = [
    "name","amenity","drinking_water","access","fee","opening_hours",
    "wheelchair","operator","website","mapillary","toilets:wheelchair",
    "toilets:disposal","source","image","wikimedia_commons",
]
def _trail_buffer_4326(etapp_gdfs: dict[str, gpd.GeoDataFrame], distance_m: int, crs_m: int = 3857):
    parts = []
    for g in etapp_gdfs.values():
        if g is None or g.empty:
            continue
        # union on the GeoSeries explicitly
        parts.append(g[["geometry"]].to_crs(epsg=crs_m).geometry.union_all())

    if not parts:
        return shapely.geometry.GeometryCollection()

    # ⬇️ was: shapely.ops.union_all(parts)
    merged = shapely.union_all(parts)
    buf = merged.buffer(distance_m)
    return gpd.GeoSeries([buf], crs=f"EPSG:{crs_m}").to_crs(epsg=4326).iloc[0]


def _poi_bbox_for_trail(etapp_gdfs: dict[str, gpd.GeoDataFrame], margin_m: int = 1500, crs_m: int = 3857):
    """Trail-BBOX + marginal för Overpass-fråga."""
    minx, miny, maxx, maxy = _bbox_from_gdfs(etapp_gdfs)
    bb_poly = gpd.GeoSeries([shapely.geometry.box(minx, miny, maxx, maxy)], crs="EPSG:4326").to_crs(epsg=crs_m)
    bx = bb_poly.total_bounds
    bx = (bx[0]-margin_m, bx[1]-margin_m, bx[2]+margin_m, bx[3]+margin_m)
    bb2 = gpd.GeoSeries([shapely.geometry.box(*bx)], crs=f"EPSG:{crs_m}").to_crs(epsg=4326)
    return tuple(bb2.total_bounds.tolist())  # minx, miny, maxx, maxy

def fetch_amenities_in_bbox(minx: float, miny: float, maxx: float, maxy: float) -> list[dict]:
    """
    Hämtar noder/ways/rel som är relevanta för toaletter och dricksvatten.
    Vi använder 'out center;' för att få centroid för ytor/rel.
    """
    q = f"""
    [out:json][timeout:180];
    (
      nwr["amenity"="toilets"]({miny},{minx},{maxy},{maxx});
      nwr["amenity"="drinking_water"]({miny},{minx},{maxy},{maxx});
      nwr["drinking_water"="yes"]({miny},{minx},{maxy},{maxx});
      nwr["amenity"="water_point"]({miny},{minx},{maxy},{maxx});
    );
    out tags center;
    """
    data = _overpass(q)
    return data.get("elements", [])

def pois_to_gdf(elements: list[dict]) -> gpd.GeoDataFrame:
    """
    Konverterar Overpass-element till punkter (node-lat/lon eller centerlat/centerlon).
    Behåller osm_type/osmid + nyckeltaggar för popup.
    """
    rows = []
    for el in elements:
        typ = el.get("type")
        osmid = el.get("id")
        tags = el.get("tags", {}) or {}

        # bestäm koordinat
        if typ == "node":
            lat = el.get("lat"); lon = el.get("lon")
        else:
            lat = el.get("center", {}).get("lat")
            lon = el.get("center", {}).get("lon")
        if lat is None or lon is None:
            continue

        props = {"osm_type": typ, "osmid": osmid}
        for k in POI_KEYS:
            props[k] = tags.get(k)
        # harmonisera enkel flagga "kind"
        if tags.get("amenity") == "toilets":
            kind = "toilet"
        elif tags.get("amenity") == "drinking_water" or tags.get("drinking_water") == "yes" or tags.get("amenity") == "water_point":
            kind = "water"
        else:
            continue
        props["kind"] = kind
        rows.append({**props, "geometry": shapely.geometry.Point(float(lon), float(lat))})

    if not rows:
        return gpd.GeoDataFrame(columns=["osm_type","osmid","kind","geometry"], geometry="geometry", crs="EPSG:4326")
    df  = pd.DataFrame(rows)
    gdf = gpd.GeoDataFrame(df, geometry="geometry", crs="EPSG:4326")
    return gdf

def _mk_osm_obj_links(osm_type: str, osmid: int, lat: float | None = None, lon: float | None = None) -> str:
    """Länkar för node/way/relation + Wikimap + Mapillary."""
    try:
        osmid = int(osmid)
    except Exception:
        return ""
    base = f"https://www.openstreetmap.org/{osm_type}/{osmid}"
    edit = f"https://www.openstreetmap.org/edit?editor=id&{osm_type}={osmid}"
    hist = f"https://www.openstreetmap.org/{osm_type}/{osmid}/history"
    vhis = f"https://osmlab.github.io/osm-deep-history/#/{osm_type}/{osmid}"
    wikm = f"https://wikimap.toolforge.org/?osm={osm_type}:{osmid}"
    parts = [
        f'<a href="{base}" target="_blank">OSM {osm_type} {osmid}</a>',
        f'<a href="{edit}" target="_blank">Öppna i iD</a>',
        f'<a href="{hist}" target="_blank">Historik</a>',
        f'<a href="{vhis}" target="_blank">Visual history</a>',
        f'<a href="{wikm}" target="_blank">Wikimap här</a>',
    ]
    if lat is not None and lon is not None:
        parts.append(f'<a href="https://www.mapillary.com/app/?z=18&lat={lat:.6f}&lng={lon:.6f}&focus=map" target="_blank">Mapillary här</a>')
    return " · ".join(parts)

def _poi_popup_html(row: pd.Series) -> str:
    title = row.get("name") or ("🚻 Toalett" if row.get("kind")=="toilet" else "🚰 Dricksvatten")
    # try to render a single image (keeps popup light)
    img_html = ""
    img_val  = row.get("image")
    commons  = row.get("wikimedia_commons")
    if img_val:
        img_html = make_image_html(img_val)
    elif isinstance(commons, str) and commons.lower().startswith(("file:", "image:")):
        thumb = commons_thumb_url(commons, width=520)
        link  = commons_filepage_url(commons)
        img_html = (f'<a href="{html.escape(link)}" target="_blank">'
                    f'<img src="{html.escape(thumb)}" style="max-width:100%;height:auto;border-radius:8px"/></a>')

    # Links: OSM/iD/History/Wikimap/Mapillary + Wheelmap + Commons
    lat = float(row.geometry.y) if row.geometry is not None else None
    lon = float(row.geometry.x) if row.geometry is not None else None
    links = [_mk_osm_obj_links(str(row.get("osm_type")), int(row.get("osmid")), lat, lon)]

    # Wheelmap (node & way supported)
    wm = _wheelmap_url(str(row.get("osm_type")), row.get("osmid"))
    if wm:
        links.append(f'♿ <a href="{wm}" target="_blank" rel="noopener">Wheelmap</a>')

    # Commons file page (when tag exists)
    if isinstance(commons, str) and commons.strip():
        ct = commons.replace(" ", "_")
        commons_url = f"https://commons.wikimedia.org/wiki/{urllib.parse.quote(ct, safe=':_()%/+-')}"
        links.append(f"🖼️ <a href='{commons_url}' target='_blank' rel='noopener'>Wikimedia Commons</a>")

    chips = []
    def add_chip(label, val, kind="muted"):
        if val and str(val).strip().lower() not in {"", "nan", "none"}:
            chips.append(_badge(f"{label}: {val}", kind))
    add_chip("access", row.get("access"))
    add_chip("wheelchair", row.get("wheelchair") or row.get("toilets:wheelchair"))
    add_chip("fee", row.get("fee"))
    add_chip("opening_hours", row.get("opening_hours"))
    add_chip("operator", row.get("operator"))
    if row.get("website"):
        chips.append(f'<a href="{html.escape(row["website"])}" target="_blank" style="text-decoration:none">{_badge("website","primary")}</a>')

    # hand off to your rich popup renderer
    tags = {"image": None}  # we already rendered a single image above
    extra = (
        (f'<div style="margin:8px 0">{img_html}</div>' if img_html else "") +
        f"<div style='margin-top:8px;'>{' · '.join(links)}</div>"
    )
    return build_hiking_popup_html(tags, title=title, osmid=row.get("osmid"), extra_html=extra + "".join(chips))

def build_poi_layers_for_map(m: folium.Map, etapp_gdfs: dict[str, gpd.GeoDataFrame], *, distance_m: int = 200,
                             overpass_margin_m: int = 1500):
    """Hämtar POI i bbox, filtrerar till buffert runt leden, adderar två lager med markörer."""
    if not etapp_gdfs:
        return
    # 1) Trail-buffert
    buf = _trail_buffer_4326(etapp_gdfs, distance_m=distance_m)
    if buf.is_empty:
        return
    # 2) Hämta POI i bbox
    minx, miny, maxx, maxy = _poi_bbox_for_trail(etapp_gdfs, margin_m=overpass_margin_m)
    els = fetch_amenities_in_bbox(minx, miny, maxx, maxy)
    gdf = pois_to_gdf(els)
    if gdf.empty:
        return
    # 3) Spatialt filter (inom buffert)
    within = gpd.GeoDataFrame(geometry=gpd.GeoSeries([buf], crs="EPSG:4326"))
    gdf_sel = gdf[gdf.geometry.intersects(buf)]
    if gdf_sel.empty:
        return

    # 4) Popups
    gdf_sel = gdf_sel.copy()
    gdf_sel["popup_html"] = gdf_sel.apply(_poi_popup_html, axis=1)

    # 5) Dela upp lager
    toilets = gdf_sel[gdf_sel["kind"] == "toilet"].copy()
    waters  = gdf_sel[gdf_sel["kind"] == "water"].copy()

    def _add_points_layer(df: gpd.GeoDataFrame, name: str, color: str, emoji: str):
        if df.empty: 
            return
        fg = folium.FeatureGroup(name=name, show=True)
        for _, r in df.iterrows():
            lat, lon = float(r.geometry.y), float(r.geometry.x)
            tt = r.get("name") or name
            ic = folium.Icon(color=color, icon="tint" if r["kind"]=="water" else "restroom", prefix="fa")
            folium.Marker(
                location=[lat, lon],
                icon=ic,
                tooltip=f"{emoji} {tt}",
                popup=folium.Popup(folium.IFrame(r["popup_html"], width=540, height=340), max_width=560),
            ).add_to(fg)
        fg.add_to(m)

    _add_points_layer(toilets, f"🚻 Toaletter nära leden (≤{distance_m} m)", "purple", "🚻")
    _add_points_layer(waters,  f"🚰 Dricksvatten nära leden (≤{distance_m} m)", "blue",   "🚰")

    # 6) Liten legend (hörn)
    legend = f"""
    <div style="position: fixed; bottom: 18px; right: 18px; z-index: 9999;
                background: rgba(255,255,255,0.92); padding: 10px 12px; border-radius: 10px;
                box-shadow: 0 2px 8px rgba(0,0,0,0.15); font-size: 13px;">
      <div style="font-weight:600; margin-bottom:6px;">Service nära leden (≤{distance_m} m)</div>
      <div>🚻 Toalett</div>
      <div>🚰 Dricksvatten</div>
    </div>
    """
    m.get_root().html.add_child(folium.Element(legend))


def fetch_ways_for_relation(rel_id: int) -> list[dict]:
    q = f"""
    [out:json][timeout:180];
    relation({rel_id});
    way(r);
    out tags geom;
    """
    data = _overpass(q)
    return [el for el in data.get("elements", []) if el.get("type") == "way"]

def fetch_way(way_id: int) -> list[dict]:
    q = f"""
    [out:json][timeout:180];
    way({way_id});
    out tags geom;
    """
    data = _overpass(q)
    return [el for el in data.get("elements", []) if el.get("type") == "way"]

# =================== Konvertering: Overpass -> GeoDataFrame =================
_CORE_TAGS = [
    "name","highway","surface","tracktype","foot","bicycle",
    "sac_scale","trail_visibility","smoothness","width",
    "mtb:scale","mtb:scale:imba","osmc:symbol","image",
    "incline","lit","step_count","ford","bridge","tunnel","wheelchair","ramp",
    "source","mapillary","scenic",
]

def _way_to_feature(el: dict) -> dict | None:
    if "geometry" not in el: return None
    coords = [(pt["lon"], pt["lat"]) for pt in el["geometry"]]
    if len(coords) < 2: return None
    tags = el.get("tags", {}) or {}
    props = {"osmid": el["id"]}
    for k in _CORE_TAGS:
        props[k] = tags.get(k)
    return {"properties": props, "geometry": LineString(coords)}

def elements_to_gdf(elements: list[dict]) -> gpd.GeoDataFrame:
    feats = []
    for el in elements:
        f = _way_to_feature(el)
        if f: feats.append(f)
    if not feats:
        return gpd.GeoDataFrame(columns=["osmid",*_CORE_TAGS,"geometry"], geometry="geometry", crs="EPSG:4326")
    df  = pd.DataFrame([f["properties"] for f in feats])
    gdf = gpd.GeoDataFrame(df, geometry=[f["geometry"] for f in feats], crs="EPSG:4326")
    for k in ["osmid",*_CORE_TAGS]:
        if k not in gdf.columns: gdf[k] = None
    return gdf

# =================== Bygg seg_gdf_4326 per etapp ============================
def build_sat_seg_gdfs(trail_qid: str = "Q131318799", prefer_relation: bool = True) -> dict[str, gpd.GeoDataFrame]:
    meta = fetch_sat_sections_from_wikidata(trail_qid)
    if meta.empty:
        print("Hittade inga etapper i Wikidata för", trail_qid)
        return {}

    out: dict[str, gpd.GeoDataFrame] = {}
    for _, row in meta.iterrows():
        label  = str(row["sectionLabel"]).strip() or row["section_qid"]
        rel_id = row["osmr"] if pd.notna(row["osmr"]) else None
        way_id = row["osmw"] if pd.notna(row["osmw"]) else None

        elements: list[dict] = []
        if prefer_relation and rel_id:
            try:
                elements = fetch_ways_for_relation(int(rel_id))
            except Exception as e:
                print(f"[{label}] Fel vid hämtning av relation {rel_id}: {e}")

        if (not elements) and way_id:
            try:
                elements = fetch_way(int(way_id))
            except Exception as e:
                print(f"[{label}] Fel vid hämtning av way {way_id}: {e}")

        gdf = elements_to_gdf(elements)
        if "name" not in gdf.columns:
            gdf["name"] = None
        gdf["name"] = gdf["name"].fillna(label)
        out[label] = gdf

    return out

# =================== Wiki / Taginfo länkar ==================================
def _wiki_link_for_key(key: str) -> str:
    url = WIKI_KEY_URLS.get(key)
    if not url: return ""
    return (f'<a href="{html.escape(url)}" target="_blank" '
            f'style="display:inline-block;padding:2px 8px;border-radius:999px;'
            f'font-size:12px;line-height:18px;background:#1f6feb;color:white;'
            f'margin:0 6px 6px 0;white-space:nowrap;text-decoration:none;">'
            f'Wiki: {html.escape(key)}</a>')

def _taginfo_link_for_key(key: str) -> str:
    url = f"https://taginfo.openstreetmap.org/keys/{quote(key, safe='')}"
    return (f'<a href="{html.escape(url)}" target="_blank" '
            f'style="display:inline-block;padding:2px 8px;border-radius:999px;'
            f'font-size:12px;line-height:18px;background:#6e7781;color:white;'
            f'margin:0 6px 6px 0;white-space:nowrap;text-decoration:none;">'
            f'Taginfo: {html.escape(key)}</a>')

# =================== EN karta med GLOBALE lager per kategori ================
def make_all_in_one_map_global(etapp_gdfs: dict[str, gpd.GeoDataFrame], mode: str = "foot",
                               OUTPUT_DIR: str = OUTPUT_DIR, PROJECT_NAME: str = PROJECT_NAME,
                               stamp: str = stamp, include_tracks_compacted: bool = True,
                               include_amenities: bool = True, amenity_distance_m: int = 200) -> str:
    

    bounds = []
    acc = {
        "missing_surface":   [], "missing_access":   [], "missing_hike":    [],
        "inconsistent":      [], "missing_stepcount":[], "background":      [],
        "all_segments":      [], "missing_mtbscale": [], "missing_tracktype":[],
        "scenic":            [],
    }
    if include_tracks_compacted:
        acc["tracks_compacted"] = []

    for etapp_label, seg_gdf_4326 in etapp_gdfs.items():
        if seg_gdf_4326 is None or seg_gdf_4326.empty:
            continue

        minx, miny, maxx, maxy = seg_gdf_4326.to_crs(epsg=4326).total_bounds
        bounds.append((minx, miny, maxx, maxy))

        sg = seg_gdf_4326.copy()
        for col in ["highway","surface","foot","bicycle","sac_scale","trail_visibility",
                    "tracktype","smoothness","width","mtb:scale","mtb:scale:imba",
                    "osmc:symbol","incline","lit","step_count","ford","bridge","tunnel","wheelchair","ramp","source"]:
            if col not in sg.columns:
                sg.loc[:, col] = None
            sg.loc[:, col] = sg[col].astype(str).str.lower().replace({"none": ""})

        missing_surface = sg.loc[(sg["surface"] == "") | (sg["surface"].isna())].copy()
        if mode == "foot":
            miss_access = sg.loc[(sg["highway"].isin(["path","footway"])) & (sg["foot"].isin(["","nan"]) | sg["foot"].isna())].copy()
        else:
            miss_access = sg.loc[(sg["bicycle"].isin(["","nan"]) | sg["bicycle"].isna())].copy()

        miss_hike = sg.loc[
            sg["highway"].isin(["path","footway"]) &
            ((sg["sac_scale"].isin(["","nan"]) | sg["sac_scale"].isna()) |
             (sg["trail_visibility"].isin(["","nan"]) | sg["trail_visibility"].isna()))
        ].copy()

        inconsistent = sg.loc[
            (sg["tracktype"].isin(["grade1","grade2","grade3","grade4","grade5"])) &
            (sg["highway"] != "track")
        ].copy()

        missing_stepcount = sg.loc[(sg["highway"] == "steps") & (sg["step_count"].isin(["","nan"]) | sg["step_count"].isna())].copy()
        missing_mtbscale  = sg.loc[sg["highway"].isin(["path","track"]) & (sg["mtb:scale"].isin(["","nan"]) | sg["mtb:scale"].isna()) &
                                   (sg["mtb:scale:imba"].isin(["","nan"]) | sg["mtb:scale:imba"].isna())].copy()
        missing_tracktype = sg.loc[(sg["highway"] == "track") & (sg["tracktype"].isin(["","nan"]) | sg["tracktype"].isna())].copy()
        has_scenic        = sg.loc[(sg["scenic"] == "yes")].copy()

        # name-fyllning
        for df in [missing_surface, miss_access, miss_hike, inconsistent, missing_stepcount, missing_mtbscale]:
            if "name" not in df.columns:
                df.loc[:, "name"] = etapp_label
            else:
                df.loc[:, "name"] = df["name"].fillna(etapp_label)

        acc["missing_surface"].append(missing_surface)
        acc["missing_access"].append(miss_access)
        acc["missing_hike"].append(miss_hike)
        acc["inconsistent"].append(inconsistent)
        acc["missing_stepcount"].append(missing_stepcount)
        acc["background"].append(seg_gdf_4326.loc[:, ["geometry"]].copy())
        acc["missing_mtbscale"].append(missing_mtbscale)
        acc["missing_tracktype"].append(missing_tracktype)
        acc["scenic"].append(has_scenic)

        if include_tracks_compacted:
            tracks_compacted = sg.loc[(sg["highway"] == "track") & (sg["surface"] == "compacted")].copy()
            if "name" not in tracks_compacted.columns:
                tracks_compacted.loc[:, "name"] = etapp_label
            else:
                tracks_compacted.loc[:, "name"] = tracks_compacted["name"].fillna(etapp_label)
            acc["tracks_compacted"].append(tracks_compacted)

        seg_full = seg_gdf_4326.copy()
        if "name" not in seg_full.columns:
            seg_full["name"] = etapp_label
        else:
            seg_full["name"] = seg_full["name"].fillna(etapp_label)
        acc["all_segments"].append(seg_full)

    # Init-karta
    if not bounds:
        m = folium.Map(location=[59.3293, 18.0686], zoom_start=9, control_scale=True, tiles="OpenStreetMap")
    else:
        minx = min(b[0] for b in bounds); miny = min(b[1] for b in bounds)
        maxx = max(b[2] for b in bounds); maxy = max(b[3] for b in bounds)
        m = folium.Map(location=[(miny+maxy)/2, (minx+maxx)/2], zoom_start=10, control_scale=True, tiles="OpenStreetMap")

    # Popup-hjälpare för rader
    def _build_popup_html_from_row(row):
        tags = {k: row.get(k) for k in [
            "highway","foot","bicycle","sac_scale","trail_visibility","surface","tracktype","smoothness","width",
            "mtb:scale","mtb:scale:imba","osmc:symbol","image","incline","lit","step_count","ford","bridge",
            "tunnel","wheelchair","ramp","source","mapillary"
        ]}
        title = row.get("name") or f"Way {row.get('osmid')}"
        links = _mk_all_links(row)
        extra = f'<div style="margin-top:8px;">{links}</div>'
        return build_hiking_popup_html(tags, title=title, osmid=row.get("osmid"), extra_html=extra)

    # Global lager-hjälpare
    def _make_global_layer(df_list, layer_name, color_hex, reason_text=None):
        dfs = [d for d in df_list if (d is not None and not d.empty)]
        if not dfs: return None
        df = pd.concat(dfs, ignore_index=True).copy()
        if reason_text: df.loc[:, "audit_reason"] = reason_text
        df.loc[:, "popup_html"] = df.apply(_build_popup_html_from_row, axis=1)

        fields = ["geometry", "popup_html", "highway", "surface", "tracktype"]
        tooltip_fields  = ["highway","surface","tracktype"]
        tooltip_aliases = ["highway","surface","tracktype"]
        if reason_text:
            fields.append("audit_reason"); tooltip_fields.append("audit_reason"); tooltip_aliases.append("varför i detta lager")

        gdf = gpd.GeoDataFrame(df[fields], geometry="geometry", crs="EPSG:4326")
        gj = GeoJson(
            data=gdf, name=layer_name,
            style_function=lambda _f, _c=color_hex: {"weight": 6, "color": _c},
            tooltip=GeoJsonTooltip(fields=tooltip_fields, aliases=tooltip_aliases, sticky=True),
            popup=GeoJsonPopup(fields=["popup_html"], aliases=[""], parse_html=True, labels=False, max_width=520)
        )
        fg = folium.FeatureGroup(name=layer_name)
        gj.add_to(fg)
        return fg

    # Globala lager
    L1 = _make_global_layer(acc["missing_surface"],   "Saknar surface", AUDIT_COLORS["missing_surface"])
    L2 = _make_global_layer(acc["missing_access"],    f"Saknar {'foot' if mode=='foot' else 'bicycle'}", AUDIT_COLORS["missing_access"])
    L3 = _make_global_layer(acc["missing_hike"],      "Saknar sac/visibility", AUDIT_COLORS["missing_hike"],
                            "highway=path/footway och saknar sac_scale eller trail_visibility")
    L4 = _make_global_layer(acc["inconsistent"],      "Inkonsekvent tracktype", AUDIT_COLORS["inconsistent"])
    L5 = _make_global_layer(acc["missing_stepcount"], "Saknar step_count (trappor)", AUDIT_COLORS["missing_stepcount"])
    L6 = _make_global_layer(acc["missing_mtbscale"],  "Saknar mtb:scale/mtb:scale:imba", AUDIT_COLORS["missing_mtbscale"],
                            "highway=path/track saknar mtb:scale och mtb:scale:imba")
    L7 = _make_global_layer(acc["missing_tracktype"], "Saknar tracktype (på highway=track)", AUDIT_COLORS["missing_tracktype"],
                            "highway=track men saknar tracktype")
    L8 = _make_global_layer(acc["scenic"],            "Scenic (scenic=yes)", AUDIT_COLORS["scenic"], "scenic=yes")

    for L in [L1, L2, L3, L4, L5, L6, L7, L8]:
        if L is not None: L.add_to(m)

    if include_tracks_compacted and "tracks_compacted" in acc:
        LT = _make_global_layer(acc["tracks_compacted"], "Tracks (surface=compacted)", AUDIT_COLORS["tracks_compacted"])
        if LT is not None: LT.add_to(m)

    if acc["background"]:
        bg = pd.concat(acc["background"], ignore_index=True)
        GeoJson(bg, name="Etapper (bakgrund)",
                style_function=lambda _f: {"weight": 3, "color": "#666", "opacity": 0.6}).add_to(m)
    # Alla segment (popup) + hover-bild (endast image, ingen text/Mapillary-fallback)
    if acc["all_segments"]:
        df_all = pd.concat(acc["all_segments"], ignore_index=True).copy()
    
        # säkerställ kolumner
        for col in ["highway","foot","bicycle","sac_scale","trail_visibility","surface",
                    "tracktype","smoothness","width","mtb:scale","mtb:scale:imba",
                    "osmc:symbol","image","incline","lit","step_count","ford","bridge",
                    "tunnel","wheelchair","ramp","source","name","osmid","geometry","scenic"]:
            if col not in df_all.columns:
                df_all[col] = None
    
        # popup html som tidigare
        df_all["popup_html"] = df_all.apply(_build_popup_html_from_row, axis=1)
    
        # hover: bygg en ren bild-url från 'image' (ingen fallback, ingen text)
        df_all["hover_img"] = df_all["image"].apply(lambda v: first_thumb_url(v, width=320))
    
        gdf_all = gpd.GeoDataFrame(
            df_all[["geometry","popup_html","hover_img"]],
            geometry="geometry", crs="EPSG:4326"
        )
    
        # Spara lagrets variabelnamn för att kunna koppla JS-händelser
        gj_all = GeoJson(
            data=gdf_all,
            name="Alla segment (popup)",
            style_function=lambda _f: {"weight": 3, "color": "#555", "opacity": 0.9},
            popup=GeoJsonPopup(fields=["popup_html"], aliases=[""], parse_html=True, labels=False, max_width=520)
        )
        gj_all.add_to(m)
    
        # Litet hover-fönster som följer musen
        m.get_root().html.add_child(folium.Element(f"""
    <div id="sat-hover-preview" style="position: fixed; display: none; pointer-events: none; z-index: 10000;
      border: 1px solid #ccc; background: #fff; padding: 3px; border-radius: 6px; box-shadow: 0 1px 6px rgba(0,0,0,.2);">
      <img id="sat-hover-img" style="max-width: 220px; max-height: 160px; display: block;" />
    </div>
    <script>
    (function(){{
      var layer = {gj_all.get_name()};
      var box = document.getElementById('sat-hover-preview');
      var img = document.getElementById('sat-hover-img');
    
      function move(ev){{
        var e = ev.originalEvent || ev;
        var x = e.clientX || 0;
        var y = e.clientY || 0;
        box.style.left = (x + 16) + 'px';
        box.style.top  = (y + 16) + 'px';
      }}
      function show(url, ev){{
        if(!url) return;
        if (img.getAttribute('src') !== url) img.setAttribute('src', url);
        box.style.display = 'block';
        move(ev);
      }}
      function hide(){{
        box.style.display = 'none';
        img.removeAttribute('src');
      }}
    
      // Bind mouse events per feature — bara om vi har hover_img
      layer.eachLayer(function(l){{
        var props = l && l.feature && l.feature.properties;
        if (props && props.hover_img){{
          l.on('mouseover', function(ev){{ show(props.hover_img, ev); }});
          l.on('mousemove', move);
          l.on('mouseout', hide);
        }}
      }});
    }})();
    </script>
        """))
    
        # === Toaletter & Dricksvatten nära leden ===
        if include_amenities:
            try:
                build_poi_layers_for_map(m, etapp_gdfs, distance_m=amenity_distance_m, overpass_margin_m=1500)
            except Exception as e:
                print("POI-lager (toaletter/dricksvatten) misslyckades:", e)

        add_audit_legend(m, include_tracks=include_tracks_compacted)
        folium.LayerControl(collapsed=False).add_to(m)
    
        # === Way index (ID-lista) ===============================================
        if acc.get("all_segments"):
            df_idx = pd.concat(acc["all_segments"], ignore_index=True).copy()
            needed_cols = ["osmid","name","highway","surface","foot","bicycle",
                           "sac_scale","trail_visibility","tracktype","step_count",
                           "mtb:scale","mtb:scale:imba"]
            for c in needed_cols:
                if c not in df_idx.columns: df_idx[c] = None
    
            def _is_missing(v):
                s = str(v).strip().lower() if v is not None else ""
                return s in {"", "nan", "none"}
    
            def _todo(row):
                todo = []
                hwy = str(row.get("highway") or "").lower()
                if _is_missing(row.get("surface")): todo.append("add surface")
                if mode == "foot":
                    if hwy in {"path","footway"} and _is_missing(row.get("foot")): todo.append("add foot")
                else:
                    if _is_missing(row.get("bicycle")): todo.append("add bicycle")
                if hwy in {"path","footway"} and (_is_missing(row.get("sac_scale")) or _is_missing(row.get("trail_visibility"))):
                    todo.append("add sac/visibility")
                tt = str(row.get("tracktype") or "").lower()
                if tt in {"grade1","grade2","grade3","grade4","grade5"} and hwy != "track":
                    todo.append("fix tracktype vs highway")
                if hwy == "steps" and _is_missing(row.get("step_count")):
                    todo.append("add step_count")
                if hwy in {"path","track"} and _is_missing(row.get("mtb:scale")) and _is_missing(row.get("mtb:scale:imba")):
                    todo.append("add mtb:scale/imba")
                if hwy == "track" and _is_missing(row.get("tracktype")):
                    todo.append("add tracktype")
                return ", ".join(todo)
    
            df_idx = df_idx[df_idx["osmid"].notnull()].copy()
            if not df_idx.empty:
                df_idx["osmid"] = df_idx["osmid"].astype(int)
                df_idx["what_to_do"] = df_idx.apply(_todo, axis=1)
                df_idx["name"] = df_idx["name"].fillna("Okänd etapp")
                df_idx.sort_values(["name","osmid"], inplace=True)
    
                def _idx_table_html(df):
                    rows, current_section = [], None
                    for _, r in df.iterrows():
                        sec = str(r["name"])
                        if sec != current_section:
                            current_section = sec
                            rows.append(f"<tr class='section'><td colspan='4' style='padding:10px 8px;font-weight:700;background:#f8fafc;border-top:1px solid #eee;'>{html.escape(sec)}</td></tr>")
                        osm_id = int(r["osmid"])
                        view  = f"https://www.openstreetmap.org/way/{osm_id}"
                        edit  = f"https://www.openstreetmap.org/edit?editor=id&way={osm_id}"
                        todo  = html.escape(r.get("what_to_do") or "")
                        rows.append(
                            "<tr>"
                            f"<td style='padding:8px;border-bottom:1px solid #f1f5f9;'><a href='{view}' target='_blank'>{osm_id}</a></td>"
                            f"<td style='padding:8px;border-bottom:1px solid #f1f5f9;'><a href='{edit}' target='_blank'>iD</a></td>"
                            f"<td style='padding:8px;border-bottom:1px solid #f1f5f9;'>{html.escape(sec)}</td>"
                            f"<td style='padding:8px;border-bottom:1px solid #f1f5f9;'>{todo}</td>"
                            "</tr>"
                        )
                    return "\n".join(rows)
    
                table_html = f"""
                <div id="way-index" style="margin: 24px auto; max-width: 1100px;
                     font-family: system-ui, -apple-system, Segoe UI, Roboto, Ubuntu, Helvetica, Arial, sans-serif;">
                  <h2 style="margin:0 0 8px 0;">Way Index</h2>
                  <div style="font-size:13px; color:#57606a; margin-bottom:8px">
                    ID klickable | ID editor | Section | what to do
                  </div>
                  <input id="wayFilter" placeholder="Filter (id, section, todo...)"
                         style="padding:8px 10px; width:100%; box-sizing:border-box; margin-bottom:8px;
                                border-radius:8px; border:1px solid #e5e7eb;">
                  <div style="overflow:auto; max-height: 50vh; border:1px solid #e5e7eb; border-radius:10px;">
                    <table style="width:100%; border-collapse:collapse; font-size:14px;">
                      <thead style="position: sticky; top: 0; background:#fafafa; z-index: 1;">
                        <tr>
                          <th style="text-align:left; padding:8px; border-bottom:1px solid #eee;">ID</th>
                          <th style="text-align:left; padding:8px; border-bottom:1px solid #eee;">Edit</th>
                          <th style="text-align:left; padding:8px; border-bottom:1px solid #eee;">Section</th>
                          <th style="text-align:left; padding:8px; border-bottom:1px solid #eee;">What to do</th>
                        </tr>
                      </thead>
                      <tbody id="wayIndexBody">
                        {_idx_table_html(df_idx)}
                      </tbody>
                    </table>
                  </div>
                </div>
                <script>
                  (function(){{
                    var filter = document.getElementById('wayFilter');
                    if(!filter) return;
                    filter.addEventListener('input', function() {{
                      var q = this.value.toLowerCase();
                      var rows = document.querySelectorAll('#wayIndexBody tr');
                      rows.forEach(function(r){{
                        if (r.classList.contains('section')) {{
                          r.style.display = (q=='' ? '' : 'none');
                          return;
                        }}
                        var text = r.innerText.toLowerCase();
                        r.style.display = text.indexOf(q) !== -1 ? '' : 'none';
                      }});
                    }});
                  }})();
                </script>
                """
                jump_btn = """
                <a href="#way-index" title="Hoppa till ID-listan"
                   style="position: fixed; bottom: 18px; right: 18px; z-index: 9999; background: #111827; color: white;
                          padding: 10px 12px; border-radius: 999px; text-decoration: none;
                          box-shadow: 0 2px 8px rgba(0,0,0,0.25); font-size: 13px;">
                  ID-lista ⤵
                </a>
                """
                m.get_root().html.add_child(folium.Element(jump_btn))
                m.get_root().html.add_child(folium.Element(table_html))
    
        # Fit bounds
        if bounds:
            m.fit_bounds([[min(b[1] for b in bounds), min(b[0] for b in bounds)],
                          [max(b[3] for b in bounds), max(b[2] for b in bounds)]])
    
        os.makedirs(OUTPUT_DIR, exist_ok=True)
        html_path = os.path.join(OUTPUT_DIR, f"{PROJECT_NAME}_ALL_{stamp}.html")
        add_about_box(m, issue_number=142, map_name="Todo")
        save_with_latest(m, html_path)
        print("Samlad karta (globala lager) sparad till:", html_path)
    return html_path

# =================== Split TODO-karta =======================================
def make_split_todo_map(etapp_gdfs: dict[str, gpd.GeoDataFrame], mode: str = "foot",
                        OUTPUT_DIR: str = OUTPUT_DIR, PROJECT_NAME: str = PROJECT_NAME, stamp: str = stamp) -> str:

    dfs = []
    for etapp_label, gdf in etapp_gdfs.items():
        if gdf is None or gdf.empty: continue
        gf = gdf.copy()
        gf["name"] = gf.get("name", pd.Series([None]*len(gf))).fillna(etapp_label)
        dfs.append(gf)
    if not dfs:
        print("Inga segment att visa."); return ""

    df_all = pd.concat(dfs, ignore_index=True).copy()
    required = ["osmid","geometry","name","highway","surface","foot","bicycle",
                "sac_scale","trail_visibility","tracktype","step_count",
                "mtb:scale","mtb:scale:imba","smoothness","width","osmc:symbol",
                "image","incline","lit","ford","bridge","tunnel","wheelchair","ramp","source","mapillary"]
    for c in required:
        if c not in df_all.columns: df_all[c] = None

    def _norm(s): return s.astype(str).str.lower().replace({"none": ""})
    for col in ["highway","surface","foot","bicycle","sac_scale","trail_visibility","tracktype","step_count","mtb:scale","mtb:scale:imba"]:
        df_all[col] = _norm(df_all[col])

    def _build_popup_html_from_row(row):
        tags = {k: row.get(k) for k in [
            "highway","foot","bicycle","sac_scale","trail_visibility","surface","tracktype","smoothness","width",
            "mtb:scale","mtb:scale:imba","osmc:symbol","image","incline","lit","step_count","ford","bridge",
            "tunnel","wheelchair","ramp","source","mapillary"
        ]}
        title = row.get("name") or f"Way {row.get('osmid')}"
        links = _mk_all_links(row)
        extra = f'<div style="margin-top:8px;">{links}</div>'
        return build_hiking_popup_html(tags, title=title, osmid=row.get("osmid"), extra_html=extra)

    df_all["popup_html"] = df_all.apply(_build_popup_html_from_row, axis=1)

    def _is_missing(series: pd.Series) -> pd.Series:
        s = series.fillna("").astype(str).str.strip().str.lower()
        return (s == "") | (s == "nan")

    H = df_all["highway"]
    m_surface   = _is_missing(df_all["surface"])
    if mode == "foot":
        m_access = (H.isin(["path","footway"])) & _is_missing(df_all["foot"]); access_label = "add foot"
    else:
        m_access = _is_missing(df_all["bicycle"]); access_label = "add bicycle"
    m_hike      = H.isin(["path","footway"]) & (_is_missing(df_all["sac_scale"]) | _is_missing(df_all["trail_visibility"]))
    m_incon_tt  = df_all["tracktype"].isin(["grade1","grade2","grade3","grade4","grade5"]) & (H != "track")
    m_step      = (H == "steps") & _is_missing(df_all["step_count"])
    m_mtb       = H.isin(["path","track"]) & _is_missing(df_all["mtb:scale"]) & _is_missing(df_all["mtb:scale:imba"])
    m_tt_on_trk = (H == "track") & _is_missing(df_all["tracktype"])

    todo_specs = [
        ("add surface",                 m_surface,   AUDIT_COLORS["missing_surface"]),
        (access_label,                  m_access,    AUDIT_COLORS["missing_access"]),
        ("add sac/visibility",          m_hike,      AUDIT_COLORS["missing_hike"]),
        ("fix tracktype vs highway",    m_incon_tt,  AUDIT_COLORS["inconsistent"]),
        ("add step_count",              m_step,      AUDIT_COLORS["missing_stepcount"]),
        ("add mtb:scale/imba",          m_mtb,       AUDIT_COLORS["missing_mtbscale"]),
        ("add tracktype",               m_tt_on_trk, AUDIT_COLORS["missing_tracktype"]),
    ]

    gdf_all = gpd.GeoDataFrame(df_all, geometry="geometry", crs="EPSG:4326")
    if gdf_all.empty:
        m2 = folium.Map(location=[59.3293, 18.0686], zoom_start=9, control_scale=True, tiles="OpenStreetMap")
    else:
        minx, miny, maxx, maxy = gdf_all.total_bounds
        m2 = folium.Map(location=[(miny+maxy)/2, (minx+maxx)/2], zoom_start=10, control_scale=True, tiles="OpenStreetMap")

    GeoJson(data=gdf_all[["geometry"]], name="Etapper (bakgrund)",
            style_function=lambda _f: {"weight": 3, "color": "#666", "opacity": 0.5}).add_to(m2)

    def _add_todo_layer(mask: pd.Series, layer_name: str, color_hex: str):
        gsel = gdf_all.loc[mask].copy()
        if gsel.empty: return
        gsel["tooltip_section"] = gsel["name"].fillna("")
        gsel["tooltip_id"]      = gsel["osmid"].fillna("").astype(str)
        gj = GeoJson(
            data=gsel[["geometry","popup_html","tooltip_section","tooltip_id"]],
            name=f"TODO: {layer_name}",
            style_function=lambda _f, _c=color_hex: {"weight": 6, "color": _c},
            tooltip=GeoJsonTooltip(fields=["tooltip_id","tooltip_section"], aliases=["ID","Section"], sticky=True),
            popup=GeoJsonPopup(fields=["popup_html"], aliases=[""], parse_html=True, labels=False, max_width=520)
        )
        fg = folium.FeatureGroup(name=f"TODO: {layer_name}")
        gj.add_to(fg); fg.add_to(m2)
        m2.get_root().html.add_child(folium.Element(f"""
        <script>
          (function() {{
            window.__todoLayers = window.__todoLayers || [];
            window.__todoLayers.push({gj.get_name()});
          }})();
        </script>
        """))

    for lname, lmask, lcolor in todo_specs:
        _add_todo_layer(lmask, lname, lcolor)

    add_audit_legend(m2, include_tracks=False)
    folium.LayerControl(collapsed=False).add_to(m2)

    if not gdf_all.empty:
        m2.fit_bounds([[gdf_all.total_bounds[1], gdf_all.total_bounds[0]],
                       [gdf_all.total_bounds[3], gdf_all.total_bounds[2]]])

    os.makedirs(OUTPUT_DIR, exist_ok=True)
    html_path2 = os.path.join(OUTPUT_DIR, f"{PROJECT_NAME}_split_todo_{stamp}.html")
    add_about_box(m2, issue_number=142, map_name="Audit layers")
    save_with_latest(m2, html_path2)
    print("Split-todo karta sparad till:", html_path2)
    return html_path2

# =================== Image QC-karta =========================================
def make_image_qc_map(etapp_gdfs: dict[str, gpd.GeoDataFrame],
                      OUTPUT_DIR: str = OUTPUT_DIR, PROJECT_NAME: str = PROJECT_NAME, 
                      stamp: str = stamp, include_amenities: bool = False,
                      amenity_distance_m: int = 200) -> str:

    dfs = []
    for etapp_label, gdf in etapp_gdfs.items():
        if gdf is None or gdf.empty: continue
        gf = gdf.copy()
        gf["name"] = gf.get("name", pd.Series([None]*len(gf))).fillna(etapp_label)
        dfs.append(gf)
    if not dfs:
        print("Inga segment att visa (image QC)."); return ""

    df_all = pd.concat(dfs, ignore_index=True).copy()
    required = ["osmid","geometry","name","image","highway","surface","tracktype",
                "foot","bicycle","sac_scale","trail_visibility","smoothness","width",
                "mtb:scale","mtb:scale:imba","osmc:symbol","incline","lit",
                "step_count","ford","bridge","tunnel","wheelchair","ramp","source","mapillary"]
    for c in required:
        if c not in df_all.columns: df_all[c] = None

    def _build_popup_html_from_row(row):
        tags = {k: row.get(k) for k in [
            "highway","foot","bicycle","sac_scale","trail_visibility","surface","tracktype","smoothness","width",
            "mtb:scale","mtb:scale:imba","osmc:symbol","image","incline","lit","step_count","ford","bridge",
            "tunnel","wheelchair","ramp","source","mapillary"
        ]}
        title = row.get("name") or f"Way {row.get('osmid')}"
        links = _mk_all_links(row)

        extra = f'<div style="margin-top:8px;">{links}</div>'
        return build_hiking_popup_html(tags, title=title, osmid=row.get("osmid"), extra_html=extra)

    s = df_all["image"].fillna("").astype(str).str.strip()
    df_all["has_image"] = s.ne("") & s.str.lower().ne("nan")

    def _check_p10689(row):
        try:
            if not row["has_image"] or pd.isna(row["osmid"]): return False
            title = normalize_commons_title(str(row["image"]))
            if not title: return False
            ok, _ = commons_has_p10689(title, int(row["osmid"]))
            return bool(ok)
        except Exception:
            return False

    df_all["has_p10689"] = df_all.apply(_check_p10689, axis=1)
    df_all["popup_html"] = df_all.apply(_build_popup_html_from_row, axis=1)
    gdf_all = gpd.GeoDataFrame(df_all, geometry="geometry", crs="EPSG:4326")

    if gdf_all.empty:
        m3 = folium.Map(location=[59.3293, 18.0686], zoom_start=9, control_scale=True, tiles="OpenStreetMap")
    else:
        minx, miny, maxx, maxy = gdf_all.total_bounds
        m3 = folium.Map(location=[(miny+maxy)/2, (minx+maxx)/2], zoom_start=10, control_scale=True, tiles="OpenStreetMap")

    GeoJson(data=gdf_all[["geometry"]], name="Etapper (bakgrund)",
            style_function=lambda _f: {"weight": 3, "color": "#666", "opacity": 0.5}).add_to(m3) 
    
    def _add_layer(mask: pd.Series, layer_name: str, color_hex: str):
        gsel = gdf_all.loc[mask].copy()
        if gsel.empty: return
        gj = GeoJson(
            data=gsel[["geometry","popup_html","name"]],
            name=layer_name,
            style_function=lambda _f, _c=color_hex: {"weight": 6, "color": _c},
            tooltip=GeoJsonTooltip(fields=["name"], aliases=["Section"], sticky=True),
            popup=GeoJsonPopup(fields=["popup_html"], aliases=[""], parse_html=True, labels=False, max_width=520)
        )
        fg = folium.FeatureGroup(name=layer_name)
        gj.add_to(fg); fg.add_to(m3)  
        
    if include_amenities:
        try:
            build_poi_layers_for_map(
                m3,
                etapp_gdfs,
                distance_m=amenity_distance_m,
                overpass_margin_m=1500
            )
        except Exception as e:
            print("POI-lager (toaletter/dricksvatten) misslyckades:", e)

    COLOR_HAS = "#27ae60"
    COLOR_NO  = "#e67e22"
    COLOR_P89 = "#1f6feb"

    _add_layer(gdf_all["has_image"],  "✔ Has image (P18/tag)",       COLOR_HAS)
    _add_layer(~gdf_all["has_image"], "✖ Missing image",             COLOR_NO)
    _add_layer(gdf_all["has_p10689"], "✔ Image linked via P10689",   COLOR_P89)

    legend_html = f"""
    <div style="position: fixed; bottom: 18px; left: 18px; z-index: 9999;
                background: rgba(255,255,255,0.92); padding: 10px 12px; border-radius: 10px;
                box-shadow: 0 2px 8px rgba(0,0,0,0.15); font-size: 13px;">
      <div style="font-weight:600; margin-bottom:6px;">Bildkvalitet</div>
      <div><span style="display:inline-block;width:14px;height:3px;background:{COLOR_HAS};margin-right:6px;"></span>Har bild (P18/tag)</div>
      <div><span style="display:inline-block;width:14px;height:3px;background:{COLOR_P89};margin-right:6px;"></span>Bild kopplad via P10689</div>
      <div><span style="display:inline-block;width:14px;height:3px;background:{COLOR_NO};margin-right:6px;"></span>Saknar bild</div>
    </div>
    """
    m3.get_root().html.add_child(folium.Element(legend_html))
    folium.LayerControl(collapsed=False).add_to(m3)

    if not gdf_all.empty:
        m3.fit_bounds([[gdf_all.total_bounds[1], gdf_all.total_bounds[0]],
                       [gdf_all.total_bounds[3], gdf_all.total_bounds[2]]])

    os.makedirs(OUTPUT_DIR, exist_ok=True)
    html_path3 = os.path.join(OUTPUT_DIR, f"{PROJECT_NAME}_image_qc_{stamp}.html")
    add_about_box(m3, issue_number=142, map_name="Image Quality Control")
    save_with_latest(m3, html_path3)

    print("Image-QC karta sparad till:", html_path3)
    return html_path3

# =================== About-box (Notebook/Edge-safe) =========================
def add_about_box(
    m,
    issue_number: int,
    map_name: str,
    created_date: str | None = None,
    repo: str = "salgo60/Stockholm_Archipelago_Trail",
    collapsed: bool = False,
):
    """
    Notebook/Edge-säker About-box med string.Template ($-placeholder).
    """
    if created_date is None:
        created_date = datetime.now().strftime("%Y-%m-%d %H:%M")

    map_dom_id = m.get_name()
    box_id     = f"sat-about-{map_dom_id}"
    header_id  = f"{box_id}-hdr"
    issue_url  = f"https://github.com/{repo}/issues/{issue_number}"

    links = [
        ("SAT Dashboard", "https://raw.githack.com/salgo60/Stockholm_Archipelago_Trail/main/notebook/output/SAT_ALL_IN_ONE_142_3_dashboard_latest.html"),
        ("Project repo issues", "https://github.com/salgo60/Stockholm_Archipelago_Trail/issues?q=is%3Aissue"),
        ("Trail on OSM (rel 19012437)", "https://www.openstreetmap.org/relation/19012437"),
        ("Trail on Wikicommons", "https://commons.wikimedia.org/wiki/Category:Stockholm_Archipelago_Trail"),
        ("Official page", "https://stockholmarchipelagotrail.com/"),
        ("Unofficial FB group", "https://www.facebook.com/groups/2875020699552247"),
        ("Visit Sweden", "https://traveltrade.visitsweden.com/plan/news-sweden/Stockholm-Archipelago-Trail/"),
    ]
    links_html = "".join(
        f'<div><a href="{_html.escape(u)}" target="_blank" style="text-decoration:none;">🔗 {_html.escape(t)}</a></div>'
        for t, u in links
    )
    collapsed_class = "sat-about-collapsed" if collapsed else ""

    tpl = Template(r"""
<style>
  .sat-about { position: fixed; z-index: 10000; background: rgba(255,255,255,0.97);
    border: 2px solid #666; border-radius: 10px; box-shadow: 0 2px 6px rgba(0,0,0,0.25);
    font: 12px/1.35 -apple-system, system-ui, Segoe UI, Roboto, Helvetica, Arial, sans-serif;
    pointer-events: auto; min-width: 240px; max-width: 320px; }
  .sat-about-header { cursor: pointer; padding: 8px 10px; font-weight: 700;
    display: flex; align-items: center; gap: 6px; user-select: none;
    background: rgba(248,248,248,.9); border-bottom: 1px solid #e5e7eb; }
  .sat-about-body { padding: 8px 10px 10px 10px; }
  .sat-about-collapsed .sat-about-body { display: none; }
  .sat-chevron { margin-left: auto; transition: transform .15s ease-in-out; }
  .sat-about-collapsed .sat-chevron { transform: rotate(-90deg); }
  .sat-links { margin-top: 6px; padding-top: 6px; border-top: 1px solid #e5e7eb; }
  @media (max-width: 640px) { .sat-about { font-size: 11px; min-width: 200px; max-width: 260px; } }
</style>

<div id="$box_id" class="sat-about $collapsed_class">
  <div id="$header_id" class="sat-about-header" title="Click to collapse/expand">
    <span>ℹ️ About</span>
    <span class="sat-chevron">▸</span>
  </div>
  <div class="sat-about-body">
    <div style="font-weight:700;margin-bottom:4px;">Stockholm Archipelago Trail Map</div>
    <div>Issue: <a href="$issue_url" target="_blank">#$issue_number</a>&nbsp;&nbsp; Map: $map_name</div>
    <div>Created: $created_date</div>
    <div>Latest updates: saved as <i>_latest.html</i></div>
    <div class="sat-links">$links_html</div>
  </div>
</div>

<script>
(function(){
  var mapId = "$map_dom_id";
  var boxId = "$box_id";
  var hdrId = "$header_id";
  var storageKey = "satAboutCollapsed_" + "$map_dom_id" + "_#$issue_number";

  function setCollapsed(box, collapsed) {
    var body = box.querySelector(".sat-about-body");
    if (collapsed) { box.classList.add("sat-about-collapsed"); if (body) body.style.display = "none"; }
    else { box.classList.remove("sat-about-collapsed"); if (body) body.style.display = "block"; }
    try { localStorage.setItem(storageKey, collapsed ? "1" : "0"); } catch(e) {}
  }

  function placeBox(mapEl, box) {
    var zoom = mapEl.querySelector(".leaflet-control-zoom");
    var mr   = mapEl.getBoundingClientRect();
    var top = 10, left = 10;
    if (zoom) {
      var zr = zoom.getBoundingClientRect();
      top  = (zr.bottom - mr.top) + 8;
      left = (zr.left   - mr.left) + zr.width + 8;
    }
    box.style.top  = top  + "px";
    box.style.left = left + "px";
  }

  function init(tries) {
    tries = tries || 0;
    var mapEl = document.getElementById(mapId);
    var box   = document.getElementById(boxId);
    var hdr   = document.getElementById(hdrId);
    if (!mapEl || !box || !hdr) { if (tries < 60) return setTimeout(function(){ init(tries+1); }, 120); return; }

    try {
      var stored = localStorage.getItem(storageKey);
      if (stored === "1") setCollapsed(box, true);
      if (stored === "0") setCollapsed(box, false);
    } catch(e) {}

    hdr.addEventListener("click", function(e){
      e.stopPropagation();
      var willCollapse = !box.classList.contains("sat-about-collapsed");
      setCollapsed(box, willCollapse);
    });

    function doPlace(){ placeBox(mapEl, box); }
    var placeTries = 0, iv = setInterval(function(){ doPlace(); if (++placeTries > 25) clearInterval(iv); }, 120);
    window.addEventListener("resize", doPlace);
    requestAnimationFrame(doPlace);
  }

  if (document.readyState === "loading") { document.addEventListener("DOMContentLoaded", function(){ init(0); }); }
  else { init(0); }
})();
</script>
""")

    html_snippet = tpl.substitute(
        box_id=box_id,
        header_id=header_id,
        collapsed_class=collapsed_class,
        issue_url=issue_url,
        issue_number=str(issue_number),
        map_name=_html.escape(map_name),
        created_date=_html.escape(created_date),
        links_html=links_html,
        map_dom_id=map_dom_id,
    )
    m.get_root().html.add_child(folium.Element(html_snippet))

# =================== Save helpers ===========================================
def _to_latest_name(filename: str | Path) -> Path:
    p = Path(filename)
    stem = p.stem
    new_stem = re.sub(r'_(?:20\d{6})(?:_\d{4})?$', '', stem)
    return p.with_name(new_stem + "_latest.html")

def save_with_latest(map_object, filename: str | Path):
    map_object.save(str(filename))
    latest_path = _to_latest_name(filename)
    shutil.copyfile(filename, latest_path)
    print(f"Saved: {filename}\nUpdated: {latest_path}")

# === SAT WATER PROXIMITY ANALYSIS ============================================
# Drop this block after your helpers. It computes distance from SAT ways to water
# and builds both: (1) a stats table (per section + global) and (2) a Folium map
# with colorized distances + optional hotspots (farthest-from-water points).

import math
from typing import Tuple

# --- CONFIG ------------------------------------------------------------------
WATER_QUERY_MARGIN_M = 2000   # how far beyond trail bbox to fetch water (meters)
DIST_BUCKETS_M = [50, 100, 250, 500, 1000]  # distance bands in meters for stats & map
CRS_METERS = 3857  # for simplicity; swap to 3006 (SWEREF99 TM) if you prefer

# --- FETCH WATER FROM OSM (Overpass) ----------------------------------------

def _bbox_from_gdfs(etapp_gdfs: dict[str, gpd.GeoDataFrame]) -> Tuple[float,float,float,float]:
    boxes = []
    for g in etapp_gdfs.values():
        if g is None or g.empty:
            continue
        minx, miny, maxx, maxy = g.to_crs(epsg=4326).total_bounds
        boxes.append((minx, miny, maxx, maxy))
    if not boxes:
        return (18.0, 59.0, 19.0, 60.0)
    minx = min(b[0] for b in boxes); miny = min(b[1] for b in boxes)
    maxx = max(b[2] for b in boxes); maxy = max(b[3] for b in boxes)
    return (minx, miny, maxx, maxy)


def _expand_bbox_meters(bbox4326: Tuple[float,float,float,float], meters: int) -> Tuple[float,float,float,float]:
    # crude expansion using WebMerc projection
    bb_poly = gpd.GeoSeries([shapely.geometry.box(*bbox4326)], crs="EPSG:4326").to_crs(epsg=CRS_METERS)
    minx, miny, maxx, maxy = bb_poly.total_bounds
    minx -= meters; miny -= meters; maxx += meters; maxy += meters
    bb2 = gpd.GeoSeries([shapely.geometry.box(minx, miny, maxx, maxy)], crs=f"EPSG:{CRS_METERS}").to_crs(epsg=4326)
    return tuple(bb2.total_bounds.tolist())  # type: ignore


def fetch_water_in_bbox(minx: float, miny: float, maxx: float, maxy: float) -> gpd.GeoDataFrame:
    """Fetch water features relevant for proximity: coastline lines, water polygons,
    and riverbanks (as area). Returns a GeoDataFrame in EPSG:4326."""
    # natural=coastline (ways, line), natural=water (polygons), waterway=riverbank (polygons)
    # also consider water=lake on multipolygons
    q = f"""
    [out:json][timeout:180];
    (
      way["natural"="coastline"]({miny},{minx},{maxy},{maxx});
      relation["natural"="water"]({miny},{minx},{maxy},{maxx});
      way["natural"="water"]({miny},{minx},{maxy},{maxx});
      relation["waterway"="riverbank"]({miny},{minx},{maxy},{maxx});
      way["waterway"="riverbank"]({miny},{minx},{maxy},{maxx});
      relation["water"="lake"]({miny},{minx},{maxy},{maxx});
      way["water"="lake"]({miny},{minx},{maxy},{maxx});
    );
    out body;
    >; out skel qt;
    """
    data = _overpass(q)

    # Build GeoDataFrame from Overpass elements (ways + relations as ways)
    nodes = {el['id']:(el.get('lon'), el.get('lat')) for el in data.get('elements',[]) if el.get('type')=='node'}
    feats = []
    for el in data.get('elements', []):
        if el.get('type') != 'way' or 'nodes' not in el:
            continue
        coords = [nodes[nid] for nid in el['nodes'] if nid in nodes]
        if len(coords) < 2:
            continue
        tags = el.get('tags',{}) or {}
        geom = None
        if tags.get('natural') == 'coastline':
            geom = shapely.geometry.LineString(coords)
        else:
            # Try polygon if closed ring
            if coords[0] == coords[-1] and len(coords) >= 4:
                geom = shapely.geometry.Polygon(coords)
            else:
                # fallback to line
                geom = shapely.geometry.LineString(coords)
        feats.append({"geometry": geom, "natural": tags.get('natural'), "waterway": tags.get('waterway'), "water": tags.get('water')})

    if not feats:
        return gpd.GeoDataFrame(columns=["natural","waterway","water","geometry"], geometry="geometry", crs="EPSG:4326")
    gdf = gpd.GeoDataFrame(feats, geometry="geometry", crs="EPSG:4326")
    return gdf

# --- DISTANCE CALCULATION ----------------------------------------------------

def _nearest_distance_to_water(line_m: shapely.geometry.base.BaseGeometry, water_union_m: shapely.geometry.base.BaseGeometry) -> float:
    if water_union_m.is_empty:
        return math.inf
    return float(line_m.distance(water_union_m))


def summarize_water_proximity(etapp_gdfs: dict[str, gpd.GeoDataFrame]) -> pd.DataFrame:
    """Return per-section stats: length share within distance bands from water, and farthest distances."""
    if not etapp_gdfs:
        return pd.DataFrame()

    # Gather SAT lines and bounds
    lines = []
    for section, g in etapp_gdfs.items():
        if g is None or g.empty:
            continue
        gf = g.copy()
        gf["section"] = section
        # Keep only LineString
        gf = gf[gf.geometry.notnull()].copy()
        lines.append(gf[["section","geometry"]])
    if not lines:
        return pd.DataFrame()

    sat = gpd.GeoDataFrame(pd.concat(lines, ignore_index=True), geometry="geometry", crs="EPSG:4326")
    bbox = _bbox_from_gdfs({"all": sat})
    bbox_exp = _expand_bbox_meters(bbox, WATER_QUERY_MARGIN_M)

    water = fetch_water_in_bbox(*bbox_exp)

    # Reproject to meters CRS
    sat_m = sat.to_crs(epsg=CRS_METERS)
    water_m = water.to_crs(epsg=CRS_METERS)

    # Build a union of water geometries (coastline lines and water polygons)
    water_union = water_m.union_all()

    # Compute per-feature distances and lengths
    sat_m["seg_len_m"] = sat_m.length
    sat_m["dist_m"] = sat_m.geometry.apply(lambda g: _nearest_distance_to_water(g, water_union))

    # Banding
    def _band(d):
        for thr in DIST_BUCKETS_M:
            if d <= thr:
                return f"≤{thr}m"
        return f">{DIST_BUCKETS_M[-1]}m"

    sat_m["dist_band"] = sat_m["dist_m"].apply(_band)

    # Aggregate per-section
    rows = []
    for section, gsec in sat_m.groupby("section"):
        total = gsec["seg_len_m"].sum()
        shares = (gsec.groupby("dist_band")["seg_len_m"].sum() / total).to_dict()
        far_max = float(gsec["dist_m"].max())
        far_mean = float(gsec["dist_m"].mean())
        rows.append({
            "section": section,
            "total_km": total/1000,
            "max_dist_m": far_max,
            "mean_dist_m": far_mean,
            **{f"share_{k}": v for k,v in shares.items()},
        })
    out = pd.DataFrame(rows).sort_values("section")
    return out

# --- MAP: COLOR BY DISTANCE + HOTSPOTS --------------------------------------

def make_water_proximity_map(
    etapp_gdfs: dict[str, gpd.GeoDataFrame],
    OUTPUT_DIR: str = OUTPUT_DIR,
    PROJECT_NAME: str = PROJECT_NAME,
    stamp: str = stamp,
    show_hotspots: bool = True,
    hotspots_per_section: int = 3,
    include_amenities: bool = True,
    amenity_distance_m: int = 200,
) -> str:
    
    if not etapp_gdfs:
        print("No data.")
        return ""

    # Build a combined lines GDF with section
    lines = []
    for section, g in etapp_gdfs.items():
        if g is None or g.empty:
            continue
        gf = g.copy()
        gf["section"] = section
        lines.append(gf[["section","geometry","name","osmid","image","mapillary"]])
    sat = gpd.GeoDataFrame(pd.concat(lines, ignore_index=True), geometry="geometry", crs="EPSG:4326")

    bbox = _bbox_from_gdfs(etapp_gdfs)
    bbox_exp = _expand_bbox_meters(bbox, WATER_QUERY_MARGIN_M)
    water = fetch_water_in_bbox(*bbox_exp)

    # Project to meters
    sat_m = sat.to_crs(epsg=CRS_METERS)
    water_m = water.to_crs(epsg=CRS_METERS)
    water_union = water_m.union_all()

    sat_m["seg_len_m"] = sat_m.length
    sat_m["dist_m"] = sat_m.geometry.apply(lambda g: _nearest_distance_to_water(g, water_union))

    # Create bands
    def _band(d):
        for thr in DIST_BUCKETS_M:
            if d <= thr:
                return f"≤{thr}m"
        return f">{DIST_BUCKETS_M[-1]}m"
    sat_m["dist_band"] = sat_m["dist_m"].apply(_band)

    # Back to 4326 for Folium
    sat_4326 = sat_m.to_crs(epsg=4326)

    # Map init
    minx, miny, maxx, maxy = sat_4326.total_bounds
    m = folium.Map(location=[(miny+maxy)/2, (minx+maxx)/2], zoom_start=10, control_scale=True, tiles="OpenStreetMap")

    # Add water backdrop (light blue) for context
    if not water.empty:
        water4326 = water.to_crs(epsg=4326)
        GeoJson(
            data=water4326,
            name="Water",
            style_function=lambda f: {"color": "#5dade2", "weight": 1, "fillColor": "#a9d6f5", "fillOpacity": 0.35},
        ).add_to(m)

    # Colors for bands
    palette = {
        f"≤{DIST_BUCKETS_M[0]}m": "#08519c",
        f"≤{DIST_BUCKETS_M[1]}m": "#3182bd",
        f"≤{DIST_BUCKETS_M[2]}m": "#6baed6",
        f"≤{DIST_BUCKETS_M[3]}m": "#9ecae1",
        f"≤{DIST_BUCKETS_M[4]}m": "#c6dbef",
        f">{DIST_BUCKETS_M[-1]}m": "#f03b20",
    }

    # Popup builder reusing your rich popup
    def _build_popup(row):
        tags = {k: row.get(k) for k in ["image"]}  # minimal, keep popup light
        title = f"{row.get('section','')} (way {int(row['osmid']) if pd.notna(row.get('osmid')) else ''}) — dist {row['dist_m']:.0f} m"
        links = " · ".join(filter(None, [ _mk_osm_links(row.get("osmid")), _mk_mapillary_link(row) ]))
        extra = f'<div style="margin-top:6px;">{links}</div>'
        return build_hiking_popup_html(tags, title=title, osmid=row.get("osmid"), extra_html=extra)

    sat_4326["popup_html"] = sat_4326.apply(_build_popup, axis=1)

    # Add band layers
    for band, color in palette.items():
        gsel = sat_4326.loc[sat_4326["dist_band"] == band]
        if gsel.empty:
            continue
        gj = GeoJson(
            data=gsel[["geometry","popup_html","section","dist_m"]],
            name=f"Distance {band}",
            style_function=lambda _f, _c=color: {"weight": 6, "color": _c},
            tooltip=GeoJsonTooltip(fields=["section","dist_m"], aliases=["Section","Dist (m)"], sticky=True),
            popup=GeoJsonPopup(fields=["popup_html"], aliases=[""], parse_html=True, labels=False, max_width=520)
        )
        fg = folium.FeatureGroup(name=f"Distance {band}")
        gj.add_to(fg)
        fg.add_to(m)

    # Hotspots: farthest segments per section
    if show_hotspots:
        rows = []
        for sec, gsec in sat_4326.groupby("section"):
            gsec_sorted = gsec.sort_values("dist_m", ascending=False).head(int(hotspots_per_section))
            rows.append(gsec_sorted)
        if rows:
            hot = pd.concat(rows)
            gjh = GeoJson(
                data=hot[["geometry","section","dist_m","popup_html"]],
                name="Farthest hotspots",
                style_function=lambda _f: {"weight": 7, "color": "#f03b20"},
                tooltip=GeoJsonTooltip(fields=["section","dist_m"], aliases=["Section","Dist (m)"], sticky=True),
                popup=GeoJsonPopup(fields=["popup_html"], aliases=[""], parse_html=True, labels=False, max_width=520)
            )
            fg = folium.FeatureGroup(name="Farthest hotspots")
            gjh.add_to(fg); fg.add_to(m)

    # Service-lager nära leden
    if include_amenities:
        try:
            build_poi_layers_for_map(m, etapp_gdfs, distance_m=amenity_distance_m, overpass_margin_m=1500)
        except Exception as e:
            print("POI-lager (toaletter/dricksvatten) misslyckades:", e)


    # Legend
    legend_lines = "".join([f"<div><span style='display:inline-block;width:14px;height:3px;background:{c};margin-right:6px;'></span>{b}</div>" for b,c in palette.items()])
    legend_html = f"""
    <div style="position: fixed; bottom: 18px; left: 18px; z-index: 9999; background: rgba(255,255,255,0.92);
                padding: 10px 12px; border-radius: 10px; box-shadow: 0 2px 8px rgba(0,0,0,0.15); font-size: 13px;">
      <div style="font-weight:600; margin-bottom:6px;">Distance to water</div>
      {legend_lines}
    </div>
    """
    m.get_root().html.add_child(folium.Element(legend_html))

    folium.LayerControl(collapsed=False).add_to(m)

    # Fit & save
    m.fit_bounds([[sat_4326.total_bounds[1], sat_4326.total_bounds[0]],
                  [sat_4326.total_bounds[3], sat_4326.total_bounds[2]]])

    os.makedirs(OUTPUT_DIR, exist_ok=True)
    html_path = os.path.join(OUTPUT_DIR, f"{PROJECT_NAME}_water_proximity_{stamp}.html")
    add_about_box(m, issue_number=142, map_name="Water proximity")
    save_with_latest(m, html_path)
    print("Water proximity map saved:", html_path)
    return html_path

# 
#
import html
import urllib.parse

def _commons_filepath_url(title: str) -> str:
    """
    Bygger en stabil bild-URL via Commons Special:FilePath (visar originalfilen).
    title kan vara 'File:Example.jpg' eller bara 'Example.jpg'
    """
    if not title:
        return ""
    t = title.replace("File:", "").replace("file:", "")
    t = t.replace(" ", "_")
    quoted = urllib.parse.quote(t, safe="()_-%!$&',;=@")  # behåll vanliga tecken
    return f"https://commons.wikimedia.org/wiki/Special:FilePath/{quoted}"

def _wheelmap_url(osm_type: str, osm_id: str | int, wheelmap_relation_id: str | None = None) -> str:
    """
    Bygg Wheelmap-länk:
      - node -> https://wheelmap.org/node/<id>
      - way  -> https://wheelmap.org/way/<id>
      - relation -> endast om du anger ett separat wheelmap_relation_id,
                    då Wheelmap ibland använder andra (t.o.m. negativa) ID:n.
    """
    t = (osm_type or "").strip().lower()
    try:
        oid = str(int(str(osm_id)))  # normalisera
    except Exception:
        oid = str(osm_id)

    if t in ("node", "n", "point"):
        return f"https://wheelmap.org/node/{oid}"
    if t in ("way", "w", "area"):
        return f"https://wheelmap.org/way/{oid}"
    if t in ("relation", "rel", "r") and wheelmap_relation_id:
        return f"https://wheelmap.org/relation/{wheelmap_relation_id}"
    return ""


def _osm_identifier(props: dict):
    """
    Försök hitta OSM-typ och id i properties.
    Stödjer vanliga format: 'id' som 'node/123', eller '@id', 'osm_id' + 'osm_type'.
    """
    # 1) Overpass/GeoJSON '@id' som 'node/123456'
    oid = props.get("@id") or props.get("id") or ""
    if isinstance(oid, str) and ("/" in oid):
        typ, num = oid.split("/", 1)
        return typ, num
    # 2) Separata fält
    typ = props.get("osm_type") or props.get("type")
    num = props.get("osm_id") or props.get("id")
    if typ and num:
        return typ, str(num)   


# === SAT DASHBOARD: Paths vs Roads (with/without cars) =======================
# Drop this file into your script after the helpers, or import the functions.
# It builds a self-contained HTML dashboard + CSVs that quantify how much of the
# trail is on: car-roads, sidewalks/pedestrian streets, tracks (forest roads),
# and dedicated foot paths, plus surfaces and tag completeness.

import io
import os
import base64
import json
import math
import html
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt

# ---------------------- Classification helpers ------------------------------
# Corridor categories (mutually exclusive, ordered by specificity)
#   1) SIDEWALK/PEDESTRIAN: footway=sidewalk OR highway in {pedestrian}
#   2) STEPS: highway=steps
#   3) DEDICATED_PATH: highway in {path, footway, bridleway}
#   4) TRACK: highway=track
#   5) CAR_ROAD: highway in {primary..service,living_street,residential,unclassified}
#   6) OTHER: everything else (cycleway, etc.)

CAR_ROAD_HWY = {
    "motorway","trunk","primary","secondary","tertiary",
    "primary_link","secondary_link","tertiary_link",
    "unclassified","residential","service","living_street"
}

DEDICATED_PATH_HWY = {"path","footway","bridleway"}


def _norm_str(s):
    if s is None:
        return ""
    s = str(s).strip().lower()
    return "" if s in {"none","nan","<na>"} else s


def corridor_class(row) -> str:
    h = _norm_str(row.get("highway"))
    f = _norm_str(row.get("foot"))
    fw = _norm_str(row.get("footway")) if "footway" in row else ""
    # sidewalk can also be on the parent road via sidewalk=*
    sidewalk = _norm_str(row.get("sidewalk"))

    if h == "steps":
        return "STEPS"
    if fw == "sidewalk" or sidewalk in {"both","left","right","yes"}:
        return "SIDEWALK/PEDESTRIAN"
    if h == "pedestrian":
        return "SIDEWALK/PEDESTRIAN"
    if h in DEDICATED_PATH_HWY:
        return "DEDICATED_PATH"
    if h == "track":
        return "TRACK"
    if h in CAR_ROAD_HWY:
        # If explicitly foot=no, still a car road (but likely wrong in route)
        return "CAR_ROAD"
    return "OTHER"


# ---------------------- Core metrics ----------------------------------------

def _ensure_length_m(gdf: gpd.GeoDataFrame, epsg=3857) -> gpd.GeoDataFrame:
    if gdf.empty:
        gdf = gdf.copy()
        gdf["length_m"] = 0.0
        return gdf
    g = gdf.to_crs(epsg=epsg).copy()
    g["length_m"] = g.geometry.length
    return g


def build_per_way_metrics(etapp_gdfs: dict[str, gpd.GeoDataFrame]) -> gpd.GeoDataFrame:
    rows = []
    for section, g in etapp_gdfs.items():
        if g is None or g.empty:
            continue
        gf = g.copy()
        gf["section"] = section
        # add fields we may use
        for c in ["highway","surface","tracktype","foot","bicycle","footway","sidewalk",
                  "sac_scale","trail_visibility","lit","step_count"]:
            if c not in gf.columns:
                gf[c] = None
        rows.append(gf)
    if not rows:
        return gpd.GeoDataFrame(columns=["section","geometry"])  # empty

    allg = gpd.GeoDataFrame(pd.concat(rows, ignore_index=True), geometry="geometry", crs="EPSG:4326")
    allg = _ensure_length_m(allg)
    allg["corridor"] = allg.apply(corridor_class, axis=1)
    return allg


def summarize_sections(allg: gpd.GeoDataFrame) -> pd.DataFrame:
    if allg.empty:
        return pd.DataFrame()
    grp = allg.groupby("section", dropna=False)
    rows = []
    for sec, g in grp:
        total = g["length_m"].sum()
        def share(mask):
            return float(g.loc[mask, "length_m"].sum() / total) if total > 0 else 0.0
        row = {
            "section": sec,
            "total_km": total/1000,
            "share_car_road": share(g["corridor"] == "CAR_ROAD"),
            "share_sidewalk_ped": share(g["corridor"] == "SIDEWALK/PEDESTRIAN"),
            "share_dedicated_path": share(g["corridor"] == "DEDICATED_PATH"),
            "share_track": share(g["corridor"] == "TRACK"),
            "share_steps": share(g["corridor"] == "STEPS"),
            "share_other": share(g["corridor"] == "OTHER"),
            "with_sac": int(g["sac_scale"].notna().sum()),
            "with_visibility": int(g["trail_visibility"].notna().sum()),
            "lit_km": g.loc[_norm_str_series(g["lit"]).isin(["yes"]) , "length_m"].sum()/1000,
        }
        # Top surfaces (length-weighted)
        surf = (g.groupby(_norm_str_series(g["surface"]).replace("", "unknown"))
                  ["length_m"].sum().sort_values(ascending=False).head(6))
        row["surface_top"] = {k: float(v) for k, v in surf.to_dict().items()}
        rows.append(row)
    return pd.DataFrame(rows).sort_values("section")


def _norm_str_series(s: pd.Series) -> pd.Series:
    return s.fillna("").astype(str).str.strip().str.lower().replace({"none":""})


def summarize_global(allg: gpd.GeoDataFrame) -> dict:
    if allg.empty:
        return {}
    total = allg["length_m"].sum()
    def share(mask):
        return float(allg.loc[mask, "length_m"].sum() / total) if total > 0 else 0.0
    by_corridor = allg.groupby("corridor")["length_m"].sum().sort_values(ascending=False)
    by_highway  = allg.groupby(_norm_str_series(allg["highway"]).replace("", "unknown"))["length_m"].sum()
    by_surface  = allg.groupby(_norm_str_series(allg["surface"]).replace("", "unknown"))["length_m"].sum()
    return {
        "total_km": float(total/1000),
        "share_car_road": share(allg["corridor"] == "CAR_ROAD"),
        "share_sidewalk_ped": share(allg["corridor"] == "SIDEWALK/PEDESTRIAN"),
        "share_dedicated_path": share(allg["corridor"] == "DEDICATED_PATH"),
        "share_track": share(allg["corridor"] == "TRACK"),
        "share_steps": share(allg["corridor"] == "STEPS"),
        "share_other": share(allg["corridor"] == "OTHER"),
        "by_corridor_m": {k: float(v) for k,v in by_corridor.to_dict().items()},
        "by_highway_m":  {k: float(v) for k,v in by_highway.to_dict().items()},
        "by_surface_m":  {k: float(v) for k,v in by_surface.to_dict().items()},
    }


# ---------------------- Chart helpers (matplotlib -> base64) ----------------

def _png_base64(fig) -> str:
    buf = io.BytesIO()
    fig.savefig(buf, format="png", bbox_inches="tight", dpi=140)
    plt.close(fig)
    data = base64.b64encode(buf.getvalue()).decode("ascii")
    return f"data:image/png;base64,{data}"


def chart_corridor_bars(global_stats: dict) -> str:
    labels = ["CAR_ROAD","SIDEWALK/PEDESTRIAN","DEDICATED_PATH","TRACK","STEPS","OTHER"]
    vals = [global_stats.get(f"share_{k.lower().replace('/','_')}", 0.0) for k in labels]
    fig, ax = plt.subplots(figsize=(6.5, 3))
    ax.bar(labels, vals)
    ax.set_ylim(0, 1)
    ax.set_ylabel("Share of total length")
    ax.set_title("Trail by corridor type (length share)")
    ax.tick_params(axis='x', rotation=25)
    return _png_base64(fig)


def chart_length_by(key: str, mapping: dict, title: str) -> str:
    items = sorted(mapping.items(), key=lambda kv: kv[1], reverse=True)[:12]
    labels = [k for k,_ in items]
    vals_m = [v for _,v in items]
    fig, ax = plt.subplots(figsize=(6.5, 3))
    ax.bar(labels, [v/1000 for v in vals_m])
    ax.set_ylabel("km")
    ax.set_title(title)
    ax.tick_params(axis='x', rotation=25)
    return _png_base64(fig)


# ---------------------- Build HTML dashboard --------------------------------

import html


def _render_map_gallery_html(items: list[dict]) -> str:
    """
    Render a responsive gallery of map links.
    Supports optional 'issues': list[str] (GitHub issue URLs).
    """
    if not items:
        return ""

    import html as _h
    import re

    def _esc(v):  # simple esc helper
        return _h.escape("" if v is None else str(v))

    def _derive_2x(src: str) -> str:
        if not src:
            return ""
        if "400x225" in src:
            return src.replace("400x225", "800x450")
        return src.replace("_400x225.", "_800x450.")

    def _issue_badges(issues: list[str] | None) -> str:
        if not issues:
            return ""
        badges = []
        for u in issues:
            num = None
            m = re.search(r"/issues/(\d+)", u or "")
            if m:
                num = m.group(1)
            title = f"Issue #{num}" if num else "GitHub Issue"
            label = f"#{num}" if num else "Issue"
            badges.append(
                f'<a class="issue-badge" href="{_esc(u)}" target="_blank" rel="noopener" title="{_esc(title)}">{_esc(label)}</a>'
            )
        # plus a general GH icon (optional—kommentera bort nästa rad om du inte vill ha den)
        badges.insert(0, '<a class="issue-icon" href="{0}" target="_blank" rel="noopener" title="GitHub Issues"><i class="fab fa-github"></i></a>'.format(_esc(issues[0])))
        return f'<div class="issues">{ "".join(badges) }</div>'

    cards = []
    for it in items:
        title  = _esc(it.get("title", ""))
        url    = _esc(it.get("url", ""))
        img1x  = _esc(it.get("img", ""))
        img2x  = _esc(it.get("img2x", "")) or _derive_2x(img1x)
        desc   = _esc(it.get("desc", ""))
        issues = it.get("issues") if isinstance(it.get("issues"), list) else None

        if img1x:
            img_type = "image/webp" if img1x.lower().endswith(".webp") else "image/jpeg"
            media_html = f"""
              <a class="gallery-card-link" href="{url}" target="_blank" rel="noopener">
                <picture>
                  <source srcset="{img1x} 1x, {img2x} 2x" type="{img_type}">
                  <img class="gallery-media"
                       src="{img1x}"
                       srcset="{img1x} 1x, {img2x} 2x"
                       sizes="(min-width: 1024px) 400px, (min-width: 680px) 50vw, 100vw"
                       width="400" height="225"
                       loading="lazy" decoding="async"
                       alt="{title}">
                </picture>
              </a>
            """
        else:
            media_html = f"""
              <a class="gallery-fallback" href="{url}" target="_blank" rel="noopener">
                <div class="gallery-fallback-title">{title}</div>
              </a>
            """

        issues_html = _issue_badges(issues)

        card = f"""
          <article class="sat-card">
              <div class="issues">
                <a class="issue-icon" href="https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/142" target="_blank" rel="noopener" title="GitHub Issues">
                  <i class="fab fa-github"></i>
                </a>
                <a class="issue-badge" href="https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/142" target="_blank" rel="noopener" title="Issue #142">#142</a>
              </div>
            {issues_html}
            {media_html}
            <div class="gallery-meta">
              <a class="gallery-title" href="{url}" target="_blank" rel="noopener">{title}</a>
              {f'<div class="gallery-desc">{desc}</div>' if desc else ''}
            </div>
          </article>
        """
        cards.append(card)

    html_out = f"""
    <section class="sat-panel">
      <div class="gallery-header">
        <h2>Kartgalleri</h2>
        <div class="muted">Länkar till senaste skapade kartorna</div>
      </div>
      <div class="sat-gallery">
        {''.join(cards)}
      </div>
    </section>

    <style>
      .intro, .links {{
        margin: 0; padding: 0.25rem 0;
      }}
      .sat-panel {{
        background:#fff; border-radius:12px; box-shadow:0 2px 10px rgba(0,0,0,.06);
        padding:12px; margin-bottom:16px;
      }}
      .gallery-header {{ display:flex; align-items:center; gap:8px; margin-bottom:10px; }}
      .gallery-header h2 {{ margin:0; font-size:16px; }}
      .muted {{ color:#6b7280; }}

      .sat-gallery {{
        display:grid; gap:12px; grid-template-columns:1fr;
      }}
      @media (min-width:680px) {{
        .sat-gallery {{ grid-template-columns:repeat(2, minmax(0,1fr)); }}
      }}
      @media (min-width:1024px) {{
        .sat-gallery {{ grid-template-columns:repeat(3, minmax(0,1fr)); }}
      }}

      .sat-card {{
        position:relative;
        background:#fff; border-radius:12px; box-shadow:0 2px 10px rgba(0,0,0,.06);
        padding:10px; display:flex; flex-direction:column; gap:8px;
      }}

      /* Issues area (uppe till höger) */
      .issues {{
        position:absolute; top:10px; right:10px;
        display:flex; align-items:center; gap:6px; flex-wrap:wrap;
        z-index:2;
      }}
      .issue-icon {{
        font-size:18px; line-height:1; text-decoration:none; color:#374151;
      }}
      .issue-icon:hover {{ color:#111827; }}

      .issue-badge {{
        font: 600 12px/1 system-ui,-apple-system,Segoe UI,Roboto,Ubuntu,Helvetica,Arial,sans-serif;
        padding:4px 6px; border-radius:999px; background:#f3f4f6; color:#111827; text-decoration:none;
        border:1px solid #e5e7eb;
      }}
      .issue-badge:hover {{
        background:#e5e7eb;
      }}

      .gallery-card-link {{ display:block; border-radius:10px; overflow:hidden; background:#f3f4f6; }}
      .gallery-media {{
        display:block;
        width:min(100%, 400px);
        height:auto;
        aspect-ratio:16/9;
        object-fit:cover;
        border-radius:10px;
        margin:0 auto;
      }}

      .gallery-fallback {{
        display:flex; align-items:center; justify-content:center;
        height:140px; border-radius:10px; background:#111827; color:#e5e7eb; text-decoration:none;
      }}
      .gallery-fallback-title {{ font-weight:600; }}

      .gallery-meta {{ display:flex; flex-direction:column; gap:4px; }}
      .gallery-title {{ font-weight:700; color:#111827; text-decoration:none; }}
      .gallery-title:hover {{ text-decoration:underline; }}
      .gallery-desc {{ color:#6b7280; font-size:13px; }}
    </style>
    """
    return html_out


def make_sat_dashboard(
    etapp_gdfs,
    OUTPUT_DIR: str,
    PROJECT_NAME: str,
    stamp: str,
    *,
    map_gallery: list[dict] | None = None,
    intro_html: str = "",                 
    charts_blocks: list[dict] | None = None,
    summary_html: str = "",
) -> str:
    gallery_html = _render_map_gallery_html(map_gallery or [])
    intro_block  = _render_intro_html(intro_html)              
    charts_html  = _render_charts_html(charts_blocks or [])    

    html_out = f"""
<!doctype html>
<html lang="en">
<head>
  <link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/font-awesome/6.4.0/css/all.min.css">
  <meta charset="utf-8">
  <title>SAT Dashboard — where do we walk on Stockholm Archipelago Trail</title>
  <meta name="viewport" content="width=device-width, initial-scale=1"/>
  <link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/font-awesome/6.4.0/css/all.min.css">
  <style>
    body{{margin:0;padding:20px;background:#f8fafc;font:14px/1.4 system-ui,-apple-system,Segoe UI,Roboto,Ubuntu,Helvetica,Arial,sans-serif;color:#111827}}
    .sat-wrap{{max-width:1200px;margin:0 auto}}
    .sat-hero{{background:#111827;border-radius:14px;color:white;padding:16px 18px;margin-bottom:16px;display:flex;flex-wrap:wrap;align-items:center;gap:10px}}
    .sat-hero h1{{margin:0;font-size:20px}}
    .sat-hero a{{color:#a5b4fc;text-decoration:none}}
    .sat-row{{display:grid;gap:14px;grid-template-columns:1fr}}
    @media(min-width:960px){{ .sat-row{{grid-template-columns:2fr 1fr}} }}
    .sat-panel{{background:#fff;border-radius:12px;box-shadow:0 2px 10px rgba(0,0,0,.06);padding:12px}}
    .muted{{color:#6b7280}}
    .sat-card{{ position:relative; }}
    .issues{{ position:absolute; top:10px; right:10px; display:flex; gap:6px; align-items:center; z-index:2; }}
    .issue-icon{{ font-size:18px; line-height:1; color:#374151; text-decoration:none; }}
    .issue-icon:hover{{ color:#111827; }}
    .issue-badge{{
      font:600 12px/1 system-ui,-apple-system,Segoe UI,Roboto,Ubuntu,Helvetica,Arial,sans-serif;
      padding:4px 6px; border-radius:999px; background:#f3f4f6; color:#111827; text-decoration:none; border:1px solid #e5e7eb;
    }}
    .issue-badge:hover{{ background:#e5e7eb; }}
  </style>
</head>
<body>
  <div class="sat-wrap">
    <div class="sat-hero">
      <h1>SAT Dashboard — where do we walk on Stockholm Archipelago Trail</h1>
      <div class="muted" style="margin-left:auto">Updated: {html.escape(stamp)}</div>
    </div>

    {intro_block}     
    {gallery_html}

    <div class="sat-row">
      <div class="sat-panel">
        {charts_html}   <!-- NEW: chart panels with descriptions -->
      </div>
      <div class="sat-panel">
        {summary_html}
      </div>
    </div>
  </div>
  </div>
  </div>

  <footer style="text-align:center;margin-top:40px;
                 font-family:sans-serif;font-size:13px;
                 color:#555;">
    👣 Totalt antal besökare: <span id="counter-total">…</span><br>
    📅 Idag: <span id="counter-today">…</span>
  </footer>

<script>
  // Totala besök
  fetch('https://api.countapi.xyz/hit/salgo60-SAT-dashboard/visits')
    .then(res => res.json())
    .then(res => {{
      document.getElementById('counter-total').innerText = res.value;
    }});

  // Besök idag (nollställs varje 24h)
  const today = new Date().toISOString().split('T')[0];
  fetch(`https://api.countapi.xyz/hit/salgo60-SAT-dashboard/{{today}}?expire=86400`)
    .then(res => res.json())
    .then(res => {{
      document.getElementById('counter-today').innerText = res.value;
    }});
</script>
</body>
</html>
"""

    # save + latest, same as before
    out_dir = Path(OUTPUT_DIR); out_dir.mkdir(parents=True, exist_ok=True)
    html_path = out_dir / f"{PROJECT_NAME}_dashboard_{stamp}.html"
    with open(html_path, "w", encoding="utf-8") as f:
        f.write(html_out)
    latest_path = _to_latest_name(html_path)
    shutil.copyfile(html_path, latest_path)
    print(f"Saved: {html_path}\nUpdated: {latest_path}")
    return str(html_path)

def _render_intro_html(intro_html: str, mode: str = "on") -> str:
    """
    mode:
      - "off"         -> render nothing
      - "inline"      -> tiny inline link bar (no big panel, minimal spacing)
      - "collapsible" -> <details> with a 'Links' summary (closed by default)
      - "panel"       -> compact panel (if you still want a box)
    """
    print("STATUS Render Intro")
    if not intro_html or mode == "off":
        return ""

    if mode == "inline":
        return f'''
        <div class="sat-links" style="margin:4px 0 6px; font-size:.92rem;
             display:flex; flex-wrap:wrap; gap:.5rem; align-items:center;">
          {intro_html}
        </div>
        <style>
          .sat-links a {{ color:#2563eb; text-decoration:none }}
          .sat-links a:hover {{ text-decoration:underline }}
          .sat-links p {{ margin:0 }}
        </style>
        '''

    if mode == "collapsible":
        return f'''
        <details class="sat-links" style="margin:2px 0">
          <summary style="cursor:pointer; font-weight:600">Links</summary>
          <div style="margin-top:6px; display:flex; flex-wrap:wrap; gap:.5rem;">
            {intro_html}
          </div>
        </details>
        <style>
          .sat-links a {{ color:#2563eb; text-decoration:none }}
          .sat-links a:hover {{ text-decoration:underline }}
          .sat-links p {{ margin:.2rem 0 }}
        </style>
        '''

    # fallback: compact panel (kept for completeness)
    return f'''
    <section class="sat-panel" style="padding:6px 0">
      <div class="sat-intro">{intro_html}</div>
    </section>
    <style>
      .sat-intro p{{margin:.2rem 0}}
      .sat-intro a{{color:#2563eb;text-decoration:none}}
      .sat-intro a:hover{{text-decoration:underline}}
    </style>
    '''

def _render_charts_html(charts: list[dict]) -> str:
    if not charts:
        return ""
    import html as _h, re

    def _badge(url: str) -> str:
        # Try to extract issue number to show like #142
        m = re.search(r"/issues/(\d+)", url)
        label = f"#{m.group(1)}" if m else "Issue"
        u = _h.escape(url)
        return f'<a class="issue-badge" href="{u}" target="_blank" rel="noopener">{label}</a>'

    out = []
    for ch in charts:
        title  = _h.escape(ch.get("title", ""))
        body   = ch.get("body_html", "")
        issues = ch.get("issues") or []  # list of URLs (strings)
        badges = " ".join(_badge(u) for u in issues)

        out.append(f"""
        <section class="sat-panel">
          <div class="chart-head">
            <h3 class="chart-title">{title}</h3>
            {'<div class="issue-badges">'+badges+'</div>' if badges else ''}
          </div>
          <div class="chart-desc">{body}</div>
        </section>
        """)

    return """
    <div class="charts-col">
      {}
    </div>
    <style>
      .charts-col > .sat-panel + .sat-panel{{margin-top:12px}}

      .chart-head{{display:flex;align-items:center;gap:8px;flex-wrap:wrap}}
      .chart-title{{margin:0;font-size:16px;font-weight:700}}

      .issue-badge{{
        display:inline-block; padding:2px 8px; border-radius:999px;
        background:#eef2ff; color:#3730a3; font-size:12px; font-weight:600;
        text-decoration:none; border:1px solid #c7d2fe;
      }}
      .issue-badge:hover{{background:#e0e7ff}}
      .chart-desc{{color:#374151}}
      .chart-desc p{{margin:.4rem 0}}
      .chart-desc a{{color:#2563eb;text-decoration:none}}
      .chart-desc a:hover{{text-decoration:underline}}
    </style>
    """.format("".join(out))
    

# ---------------------- How to call -----------------------------------------
# allg = build_per_way_metrics(etapp_gdfs)
# per_section = summarize_sections(allg)
# global_stats = summarize_global(allg)
# dash_html = make_sat_dashboard(etapp_gdfs, OUTPUT_DIR=OUTPUT_DIR, PROJECT_NAME=PROJECT_NAME, stamp=stamp)

# =================== Körning ================================================
if __name__ == "__main__":
    # Build sections once
    etapp_gdfs = build_sat_seg_gdfs(trail_qid="Q131318799", prefer_relation=True)

    # Maps
    html_path = make_all_in_one_map_global(
        etapp_gdfs=etapp_gdfs,
        mode="foot",
        OUTPUT_DIR=OUTPUT_DIR,
        PROJECT_NAME=PROJECT_NAME,
        stamp=stamp,
        include_tracks_compacted=True,
        include_amenities=True,          
        amenity_distance_m=200           
    )


    html_path = make_all_in_one_map_global(
        etapp_gdfs=etapp_gdfs,
        mode="foot",
        OUTPUT_DIR=OUTPUT_DIR,
        PROJECT_NAME=PROJECT_NAME,
        stamp=stamp,
        include_tracks_compacted=True
    )
    print("✅ All-in-one (globala lager) =>", html_path)

    html_split = make_split_todo_map(
        etapp_gdfs=etapp_gdfs,
        mode="foot",
        OUTPUT_DIR=OUTPUT_DIR,
        PROJECT_NAME=PROJECT_NAME,
        stamp=stamp
    )
    print("✅ Split TODO =>", html_split)

    html_imgqc = make_image_qc_map(
        etapp_gdfs=etapp_gdfs,
        OUTPUT_DIR=OUTPUT_DIR,
        PROJECT_NAME=PROJECT_NAME,
        stamp=stamp,
        include_amenities=True,          
        amenity_distance_m=200,
    )
    print("✅ Image QC =>", html_imgqc)

    html_steps = make_steps_only_map(
        etapp_gdfs=etapp_gdfs,
        OUTPUT_DIR=OUTPUT_DIR,
        PROJECT_NAME=PROJECT_NAME,
        stamp=stamp
    )
    print("✅ Steps only =>", html_steps)

    # ---------- Statistics (no caas_jupyter_tools) ----------
    stats_df = summarize_sat(etapp_gdfs)
    print(stats_df.head(20))
    os.makedirs(OUTPUT_DIR, exist_ok=True)
    stats_csv = os.path.join(OUTPUT_DIR, f"{PROJECT_NAME}_per_section_stats_{stamp}.csv")
    stats_df.to_csv(stats_csv, index=False)
    print("📄 Saved per-section stats CSV:", stats_csv)

    # ---------- Water proximity ----------
    water_df = summarize_water_proximity(etapp_gdfs)
    print(water_df.head(20))
    water_csv = os.path.join(OUTPUT_DIR, f"{PROJECT_NAME}_water_proximity_stats_{stamp}.csv")
    water_df.to_csv(water_csv, index=False)
    print("📄 Saved water-proximity stats CSV:", water_csv)

    html_water = make_water_proximity_map(
        etapp_gdfs=etapp_gdfs,
        OUTPUT_DIR=OUTPUT_DIR,
        PROJECT_NAME=PROJECT_NAME,
        stamp=stamp,
        show_hotspots=True,
        hotspots_per_section=3
    )
    print("✅ Water proximity =>", html_water)




### Wikipedia koppling 

In [ ]:
# -*- coding: utf-8 -*-
# Requires: SPARQLWrapper, requests, folium, pillow, geopandas
from pathlib import Path
from SPARQLWrapper import SPARQLWrapper, JSON
import folium, html, requests
from folium.plugins import BeautifyIcon
from PIL import Image, ImageOps
from io import BytesIO

# -----------------------------
# Config
# -----------------------------
LANGS = ["sv", "en", "de", "fi", "ar", "fr","nn","no","da","es"]


ROOT = Path.cwd()
if ROOT.name == "notebook":
    OUTPUT_DIR = ROOT / "output"          # → /.../notebook/output
else:
    OUTPUT_DIR = ROOT / "notebook" / "output"  # → /.../notebook/output

IMG_SUBDIR = "poi_images"
(OUTPUT_DIR / IMG_SUBDIR).mkdir(parents=True, exist_ok=True)

OUTPUT_HTML = "SAT_ALL_IN_ONE_142_3_Wikipedia.html"  

THUMB_WIDTH = 360
THUMB_FORMAT = "WEBP"
THUMB_QUALITY = 82


# -----------------------------
# Base map
# -----------------------------
m = folium.Map(location=[59.3293, 18.0686], zoom_start=8, control_scale=True)

# Bygg SAT-etapper (geopandas GeoDataFrame)
etapp_gdfs = build_sat_seg_gdfs(trail_qid="Q131318799", prefer_relation=True)
build_poi_layers_for_map(
    m=m,
    etapp_gdfs=etapp_gdfs,
    distance_m=200,
    overpass_margin_m=1500
)

# -----------------------------
# SPARQL (Wikipedia POIs)
# -----------------------------
sparql = SPARQLWrapper(
    "https://query.wikidata.org/sparql",
    agent="SAT-Dashboard/1.0 (contact: you@example.org)"
)
sparql.setReturnFormat(JSON)

lang_list = ", ".join(f'"{l}"' for l in LANGS)
query = f"""
SELECT
  ?item ?itemLabel ?coord
  (SAMPLE(?instance) AS ?instance)
  (SAMPLE(?instanceLabel) AS ?instanceLabel)
  (SAMPLE(?img) AS ?img)
  (SAMPLE(?hexcolor) AS ?hexcolor)
  (SAMPLE(?iconname) AS ?iconname)
  (SAMPLE(?instagram) AS ?instagram)
  (SAMPLE(?instagramPlace) AS ?instagramPlace)
  (SAMPLE(?facebook) AS ?facebook)
  (SAMPLE(?facebookPlace) AS ?facebookPlace)
  (SAMPLE(?gmap) AS ?gmap)
  (SAMPLE(?website) AS ?website)
  (SAMPLE(?email) AS ?email)
  (SAMPLE(?phone) AS ?phone)
  (GROUP_CONCAT(CONCAT(?lang, "|", STR(?article)); separator=";") AS ?sitelinks)
WHERE {{
  ?item wdt:P625 ?coord ;
        wdt:P6104 wd:Q134294510 .

  OPTIONAL {{ ?item wdt:P18 ?img . }}
  OPTIONAL {{
    ?item wdt:P31 ?instance .
    OPTIONAL {{ ?instance wdt:P465 ?hexcolor . }}
    OPTIONAL {{
      ?instance p:P1343 ?stmt .
      ?stmt pq:P1476 ?iconname .
    }}
  }}

  # Social + contact
  OPTIONAL {{ ?item wdt:P2003  ?instagram . }}        # Instagram username
  OPTIONAL {{ ?item wdt:P4173  ?instagramPlace . }}   # Instagram location ID
  OPTIONAL {{ ?item wdt:P2013  ?facebook . }}         # Facebook username
  OPTIONAL {{ ?item wdt:P11705 ?facebookPlace . }}    # Facebook place/page ID
  OPTIONAL {{ ?item wdt:P3749  ?gmap . }}             # Google Maps ID
  OPTIONAL {{ ?item wdt:P856   ?website . }}          # Website
  OPTIONAL {{ ?item wdt:P968   ?email . }}            # Email
  OPTIONAL {{ ?item wdt:P1329  ?phone . }}            # Phone

  OPTIONAL {{
    ?article schema:about ?item ;
             schema:inLanguage ?lang ;
             schema:isPartOf [ wikibase:wikiGroup "wikipedia" ] .
    FILTER(?lang IN ({lang_list}))
  }}

  SERVICE wikibase:label {{ bd:serviceParam wikibase:language "sv,en,de,fi,nb,da,fr,en-gb,ar". }}
}}
GROUP BY ?item ?itemLabel ?coord
"""

sparql.setQuery(query)
data = sparql.query().convert()




In [ ]:
import geopandas as gpd
import pandas as pd
import folium
import json

# -----------------------------
# Helpers
# -----------------------------
def parse_point(wkt_point):
    try:
        inner = wkt_point.split("(")[1].split(")")[0].strip()
        x, y = inner.split()
        return float(y), float(x)
    except Exception:
        return None

def parse_sitelinks(s):
    links = {}
    if not s:
        return links
    for chunk in s.split(";"):
        if "|" in chunk:
            lang, url = chunk.split("|", 1)
            links[lang] = url
    return links

def sanitize_hex(h, default="#228b22"):
    if not h:
        return default
    h = h.strip()
    if not h.startswith("#"):
        h = "#" + h
    return h if len(h) in (4, 7) else default

# Kartographer → FA mapping (extend as you like)
KARTO_TO_FA = {
    "landmark-JP": "info",
    "museum": "university",
    "viewpoint": "binoculars",
    "information": "info",
    "religious-christian": "crosshairs",
    "accommodation": "bed",
    "toilets": "male",
    "drinking-water": "tint",
    "harbor": "anchor",
    "camp-site": "tree",
}

def make_icon(symbol, hexcolor):
    fa = KARTO_TO_FA.get(symbol, "info")
    color = sanitize_hex(hexcolor, "#228b22")
    return BeautifyIcon(
        icon=fa,
        prefix="fa",
        icon_shape="marker",
        background_color=color,
        border_color="#2c3e50",
        text_color="#ffffff"
    )

def commons_fetch_bytes(p18_url, target_width):
    """Hämtar rimligt stor bild från Commons; låt Commons göra första resize."""
    if not p18_url:
        return None
    url = p18_url + (("&" if "?" in p18_url else "?") + f"width={max(int(target_width*2), 800)}")
    r = requests.get(url, timeout=25)
    r.raise_for_status()
    return r.content

def make_thumbnail_file(qid, p18_url):
    """Ladda ned bildbytes och skapa lokal tumnagel. Returnerar relativ sökväg eller None."""
    if not p18_url:
        return None
    try:
        raw = commons_fetch_bytes(p18_url, THUMB_WIDTH)
        im = Image.open(BytesIO(raw))
        im = ImageOps.exif_transpose(im)
        if im.mode not in ("RGB", "RGBA"):
            im = im.convert("RGB")
        im.thumbnail((THUMB_WIDTH, THUMB_WIDTH*10000), Image.Resampling.LANCZOS)

        ext = ".webp" if THUMB_FORMAT.upper() == "WEBP" else ".jpg"
        fname = f"{qid}{ext}"
        rel_path = Path(IMG_SUBDIR) / fname
        abs_path = OUTPUT_DIR / rel_path

        save_kwargs = {}
        if THUMB_FORMAT.upper() == "WEBP":
            save_kwargs.update(dict(format="WEBP", quality=THUMB_QUALITY, method=6))
        else:
            save_kwargs.update(dict(format="JPEG", quality=THUMB_QUALITY, optimize=True, progressive=True))

        im.save(abs_path, **save_kwargs)
        print("abs_path im:",abs_path)
        return str(rel_path).replace("\\", "/")
    except Exception:
        # Om Pillow fallerar, spara råbytes som jpg
        try:
            ext = ".jpg"
            fname = f"{qid}{ext}"
            rel_path = Path(IMG_SUBDIR) / fname
            abs_path = OUTPUT_DIR / rel_path
            with open(abs_path, "wb") as f:
                f.write(raw)
            return str(rel_path).replace("\\", "/")
        except Exception:
            return None

def commons_thumb_url(p18_url: str, width: int = 360) -> str:
    # p18_url är en Special:FilePath-URL. Lägg till width-param för skalad tumnagel.
    if not p18_url:
        return ""
    sep = "&" if "?" in p18_url else "?"
    return f"{p18_url}{sep}width={int(width)}"

def commons_file_page_url(p18_url: str) -> str:
    # Special:FilePath/Filename.ext → Commons file-sida
    if not p18_url:
        return ""
    filename = p18_url.rsplit("/", 1)[-1]
    return f"https://commons.wikimedia.org/wiki/File:{filename}"

def _clean_val(b, key):
    v = b.get(key)
    return v.get("value").strip() if isinstance(v, dict) and "value" in v else None

def _alink(label, href, title=None):
    if not href:
        return ""
    t = html.escape(title) if title else html.escape(label)
    return f'<a href="{html.escape(href)}" target="_blank" rel="noopener">{t}</a>'

def _gmap_url_from_p3749(gid: str|None) -> str|None:
    if not gid:
        return None
    gid = gid.strip()
    # If it looks like a numeric cid use ?cid=, else assume Google Place ID
    return f"https://maps.google.com/?cid={gid}" if gid.isdigit() \
           else f"https://www.google.com/maps/place/?q=place_id:{gid}"

def render_social_block(b) -> str:
    """Bygger HTML-block för sociala/länkar om de finns."""
    instagram       = _clean_val(b, "instagram")       # P2003
    instagramPlace  = _clean_val(b, "instagramPlace")  # P4173
    facebook        = _clean_val(b, "facebook")        # P2013
    facebookPlace   = _clean_val(b, "facebookPlace")   # P11705
    gmap            = _clean_val(b, "gmap")            # P3749
    website         = _clean_val(b, "website")         # P856
    email           = _clean_val(b, "email")           # P968
    phone           = _clean_val(b, "phone")           # P1329

    rows = []

    if website:
        rows.append(f'🌐 {_alink("Webbplats", website)}')

    if instagram:
        rows.append(f'📸 {_alink("Instagram", f"https://www.instagram.com/{instagram}/", "@"+instagram)}')
    if instagramPlace:
        rows.append(f'📍 {_alink("Instagram-plats", f"https://www.instagram.com/explore/locations/{instagramPlace}/")}')

    if facebook:
        rows.append(f'📘 {_alink("Facebook", f"https://www.facebook.com/{facebook}", facebook)}')
    if facebookPlace:
        rows.append(f'🏷️ {_alink("Facebook-sida", f"https://www.facebook.com/{facebookPlace}")}')

    gmaps = _gmap_url_from_p3749(gmap)
    if gmaps:
        rows.append(f'🗺️ {_alink("Google Maps", gmaps)}')

    if email:
        rows.append(f'✉️ {_alink(email, f"mailto:{email}")}')

    if phone:
        tel = "tel:" + "".join(ch for ch in phone if ch.isdigit() or ch in "+*#")
        rows.append(f'☎️ {_alink(phone, tel)}')

    if not rows:
        return ""
    return (
        "<div class='social-block' "
        "style='margin-top:8px;padding-top:6px;border-top:1px solid #eee;font-size:12px'>"
        + "<br/>".join(rows) +
        "</div>"
    )

# -----------------------------
# Build layer
# -----------------------------
fg_wp = folium.FeatureGroup(name="Wikipedia POIs (P6104 SAT)", show=True)
fg_wc   = folium.FeatureGroup(name="Toaletter", show=False)       
fg_water= folium.FeatureGroup(name="Dricksvatten", show=False)    

m.add_child(fg_wp)
m.add_child(fg_wc)
m.add_child(fg_water)
bounds_lat, bounds_lon = [], []
count_added, thumbs = 0, 0 


for b in data["results"]["bindings"]:
    pt = parse_point(b["coord"]["value"])
    if not pt:
        continue

    item_uri = b["item"]["value"]
    qid = item_uri.rsplit("/", 1)[-1]
    label = b.get("itemLabel", {}).get("value", qid)

    inst_label = b.get("instanceLabel", {}).get("value", "")
    hexcolor = b.get("hexcolor", {}).get("value")
    iconname = b.get("iconname", {}).get("value")  # Kartographer marker_symbol
    img_url = b.get("img", {}).get("value")
    sitelinks = parse_sitelinks(b.get("sitelinks", {}).get("value", ""))

    # Lokal tumnagel
    local_img_rel = make_thumbnail_file(qid, img_url)
    if local_img_rel:
        thumbs += 1

    # Språkchips
    chips = []
    for lang in LANGS:
        url = sitelinks.get(lang)
        if url:
            chips.append(
                f'<a href="{html.escape(url)}" target="_blank" rel="noopener" '
                f'style="text-decoration:none;"><span '
                f'style="border:1px solid #ddd;border-radius:12px;padding:2px 8px;'
                f'margin-right:6px;font-size:12px;">{lang}</span></a>'
            )
    chips_html = " ".join(chips) if chips else "<em>Inga artiklar i valda språk</em>"

    thumb = commons_thumb_url(img_url, THUMB_WIDTH)
    commons_page = commons_file_page_url(img_url)

    img_html = (
        f'<div style="margin-bottom:6px">'
        f'  <img src="{html.escape(thumb)}" loading="lazy" '
        f'       style="max-width:100%;height:auto;border-radius:8px">'
        f'</div>'
        f'<div style="font-size:11px;margin-top:-2px;margin-bottom:6px">'
        f'  Bild via <a href="{html.escape(commons_page)}" target="_blank" rel="noopener">Wikimedia Commons</a>'
        f'</div>'
        if img_url else ""
    )
    social_html = render_social_block(b)  # ⬅️ NEW

    popup_html = f"""
    <div style="min-width:260px">
      {img_html}
      <div style="font-weight:600;margin-bottom:4px">{html.escape(label)}</div>
      <div style="font-size:12px;margin-bottom:6px">
        <a href="https://www.wikidata.org/wiki/{qid}" target="_blank" rel="noopener">Wikidata: {qid}</a>
        {' · ' + html.escape(inst_label) if inst_label else ''}
      </div>
      <div style="margin-bottom:4px; font-size:12px">
        {'Symbol: ' + html.escape(iconname) if iconname else ''}
      </div>
      {social_html}
      <div style="margin-top:6px">{chips_html}</div>
    </div>
    """

    marker = folium.Marker(
        location=pt,
        icon=make_icon(iconname, hexcolor),
        tooltip=label,
        popup=folium.Popup(popup_html, max_width=380),
    )
    fg_wp.add_child(marker)
    bounds_lat.append(pt[0]); bounds_lon.append(pt[1])
    count_added += 1


sat_layer = folium.FeatureGroup(name="SAT-spåret", show=True)
m.add_child(sat_layer)  

# --- SAT loop (drop-in) ---
import json

sat_layer = folium.FeatureGroup(name="SAT-spåret", show=True)
m.add_child(sat_layer)

def style_fn(_):
    return {"color": "#0077ff", "weight": 4, "opacity": 0.9}

def highlight_fn(_):
    return {"weight": 6, "opacity": 1.0}

ALIASES = {
    "name": "Namn",
    "highway": "Vägtyp",
    "surface": "Yta",
    "osmid": "OSM id",
}
PREF_TOOLTIP = ["name", "highway", "surface"]
PREF_POPUP   = ["osmid", "name"]

# --- SIMPLE, ASSERTION-PROOF SAT LOOP ---
sat_layer = folium.FeatureGroup(name="SAT-spåret", show=True)
m.add_child(sat_layer)

def style_fn(_):
    return {"color": "#0077ff", "weight": 4, "opacity": 0.9}

def highlight_fn(_):
    return {"weight": 6, "opacity": 1.0}

for etapp_name, gdf in etapp_gdfs.items():
    # Ensure WGS84
    if gdf.crs and gdf.crs.to_epsg() != 4326:
        gdf = gdf.to_crs(4326)

    gj = folium.GeoJson(
        data=gdf.to_json(),           # no tooltips/popups that reference fields
        name=f"Etapp: {etapp_name}",
        style_function=style_fn,
        highlight_function=highlight_fn,
    )
    # Optional: a simple, safe tooltip that never asserts
    folium.Tooltip(etapp_name).add_to(gj)

    gj.add_to(sat_layer)

# --- end SAT loop ---


folium.LayerControl(collapsed=False).add_to(m)

#m.add_child(sat_layer)

add_about_box(m, issue_number=22, map_name="Wikipedia Wikidata kopplat till SAT") 

# Fit bounds
if bounds_lat and bounds_lon:
    sw = [min(bounds_lat), min(bounds_lon)]
    ne = [max(bounds_lat), max(bounds_lon)]
    m.fit_bounds([sw, ne])

# Save
out_path = OUTPUT_DIR / OUTPUT_HTML
m.save(str(out_path))

print(f"✅ POIs: {count_added} — Thumbnails created: {thumbs}")
print(f"🖼  Thumbs in: {OUTPUT_DIR / IMG_SUBDIR}")
print(f"💾 Map saved with SAT trail + Wikipedia POIs: {out_path}")


In [ ]:
import json
feat = json.loads(gdf.to_json())["features"][0]
print(sorted(feat.get("properties", {}).keys()))


In [ ]:
out_path

In [ ]:
OUTPUT_DIR 

In [ ]:
OUTPUT_HTML


## Skapa dashboard

In [ ]:
import html
from urllib.parse import urlparse

def _render_map_gallery_html(gallery):
    def _esc(s): 
        return html.escape(str(s), quote=True)

    def _issue_label(u):
        try:
            path = urlparse(u).path
            parts = [p for p in path.split("/") if p]
            if "issues" in parts:
                idx = parts.index("issues")
                return f"Issue #{parts[idx+1]}"
        except Exception:
            pass
        return "Länk"

    # Samla alla buzzwords
    all_buzz = []
    for item in gallery:
        buzz = item.get("buzzword", []) or item.get("buzzwords", [])
        all_buzz.extend([str(b).strip() for b in buzz])
    # Unika, sorterade buzzwords (skonsam sortering, behåll original-case)
    seen = set()
    uniq_buzz = []
    for b in all_buzz:
        key = b.casefold()
        if key not in seen:
            seen.add(key)
            uniq_buzz.append(b)
    uniq_buzz.sort(key=lambda x: x.casefold())

    def _buzz_attrs(buzz_list):
        # Data-attribut för JS-filtrering; lagra både original och normaliserad
        raw = "|".join(buzz_list)
        norm = "|".join([x.casefold() for x in buzz_list])
        return f'data-buzz="{_esc(raw)}" data-buzz-norm="{_esc(norm)}"'

    cards = []
    for item in gallery:
        title = item.get("title","")
        url = item.get("url","#")
        img = item.get("img","")
        desc = item.get("desc","")
        desc_is_html = item.get("desc_is_html", False)
        issues = item.get("issues", [])
        buzz = item.get("buzzword", []) or item.get("buzzwords", [])

        desc_html = desc if desc_is_html else _esc(desc).replace("\n","<br/>")

        buzz_html = ""
        if buzz:
            buzz_html = '<div class="buzzwrap">' + "".join(
                f'<button type="button" class="badge badge-link" data-filter-chip="{_esc(str(b))}">{_esc(str(b))}</button>'
                for b in buzz
            ) + "</div>"

        issues_html = ""
        if issues:
            issues_html = '<ul class="issues">' + "".join(
                f'<li><a href="{_esc(i)}" target="_blank" rel="noopener">{_esc(_issue_label(i))}</a></li>'
                for i in issues
            ) + "</ul>"

        card = f"""
        <article class="card" {_buzz_attrs([str(b) for b in buzz])}>
          <a class="thumb" href="{_esc(url)}" target="_blank" rel="noopener">
            <img loading="lazy" src="{_esc(img)}" alt="{_esc(title)}"/>
          </a>
          <div class="body">
            <h3 class="title"><a href="{_esc(url)}" target="_blank" rel="noopener">{_esc(title)}</a></h3>
            {buzz_html}
            <p class="desc">{desc_html}</p>
            {issues_html}
          </div>
        </article>
        """
        cards.append(card)

    # Bygg filterbaren
    filter_bar = ""
    if uniq_buzz:
        chips = "".join(
            f'<button type="button" class="chip" data-chip="{_esc(b)}">{_esc(b)}</button>'
            for b in uniq_buzz
        )
        filter_bar = f"""
        <div class="filters">
          <div class="chips" id="buzzChips">{chips}</div>
          <div class="actions">
            <button type="button" id="clearFilters" class="clear">Rensa filter</button>
          </div>
        </div>
        """

    css = """
    <style>
      .filters{display:flex;flex-wrap:wrap;align-items:center;justify-content:space-between;gap:.6rem;margin:.5rem 0 1rem}
      .chips{display:flex;flex-wrap:wrap;gap:.4rem}
      .chip{border:1px solid #ddd;background:#fafafa;border-radius:999px;padding:.28rem .65rem;font-size:.85rem;cursor:pointer}
      .chip.active{background:#111;color:#fff;border-color:#111}
      .clear{border:1px solid #ddd;background:#fff;border-radius:8px;padding:.35rem .6rem;font-size:.85rem;cursor:pointer}
      .gallery{display:grid;gap:1rem;grid-template-columns:repeat(auto-fill,minmax(320px,1fr));}
      .card{background:#fff;border:1px solid #eee;border-radius:16px;overflow:hidden;box-shadow:0 1px 3px rgba(0,0,0,.06);}
      .thumb{display:block;aspect-ratio:16/9;background:#f6f7f8;}
      .thumb img{width:100%;height:100%;object-fit:cover;display:block;}
      .body{padding:0.9rem 1rem 1rem;}
      .title{margin:.1rem 0 .4rem 0;font-size:1.05rem;line-height:1.2}
      .title a{text-decoration:none;color:#111;}
      .desc{margin:.5rem 0;color:#333}
      .buzzwrap{display:flex;flex-wrap:wrap;gap:.4rem;margin:.25rem 0 .5rem}
      .badge{display:inline-block;font-size:.78rem;padding:.18rem .5rem;border-radius:999px;border:1px solid #ddd;background:#fafafa}
      .badge-link{cursor:pointer}
      .issues{margin:.6rem 0 0 0;padding-left:1.1rem}
      .issues li{margin:.1rem 0}
      .issues a{text-decoration:none}
      .card[hidden]{display:none !important;}
      .filterinfo{font-size:.85rem;color:#555;margin:.3rem 0 .8rem}
    </style>
    """

    # Inline JS för filtrering
    js = """
    <script>
    (function(){
      const norm = s => (s || "").toLocaleLowerCase().normalize("NFKC");
      const chips = Array.from(document.querySelectorAll('[data-chip]'));
      const clearBtn = document.getElementById('clearFilters');
      const cards = Array.from(document.querySelectorAll('.card'));
      let active = new Set();

      function applyFilter(){
        if(active.size === 0){
          cards.forEach(c => c.hidden = false);
          return;
        }
        const wanted = Array.from(active).map(norm);
        cards.forEach(card=>{
          const has = (card.getAttribute('data-buzz-norm') || '');
          const tags = has.split('|').filter(Boolean);
          // OCH-logik: alla valda måste finnas på kortet
          const ok = wanted.every(w => tags.includes(w));
          card.hidden = !ok;
        });
      }

      function toggleChip(el){
        const val = el.getAttribute('data-chip');
        if(el.classList.contains('active')){
          el.classList.remove('active');
          active.delete(val);
        } else {
          el.classList.add('active');
          active.add(val);
        }
        applyFilter();
      }

      chips.forEach(ch => ch.addEventListener('click', ()=>toggleChip(ch)));

      // Klick på badge inne i kort = aktivera motsvarande chip
      document.addEventListener('click', e=>{
        const b = e.target.closest('[data-filter-chip]');
        if(!b) return;
        const val = b.getAttribute('data-filter-chip');
        const chip = chips.find(c => norm(c.getAttribute('data-chip')) === norm(val));
        if(chip) chip.click();
      });

      if(clearBtn){
        clearBtn.addEventListener('click', ()=>{
          chips.forEach(c => c.classList.remove('active'));
          active.clear();
          applyFilter();
        });
      }
    })();
    </script>
    """

    html_out = (
        css +
        filter_bar +
        '<section class="gallery">' + "\n".join(cards) + "</section>" +
        js
    )
    return html_out


In [ ]:
import html
from urllib.parse import urlparse

def _render_map_gallery_html(gallery):
    def _esc(s): 
        return html.escape(str(s), quote=True)

    def _issue_num(u: str) -> str:
        # returnerar t.ex. "#132"
        try:
            parts = [p for p in urlparse(u).path.split("/") if p]
            if "issues" in parts:
                return f"#{parts[parts.index('issues')+1]}"
        except Exception:
            pass
        return "#"

    def _issues_base(u: str) -> str:
        # bas-länk till repo-issues-sidan
        try:
            pre, post = u.split("/issues/", 1)
            return pre + "/issues"
        except ValueError:
            return u

    # --- (buzzword-kod oförändrad) ---

    cards = []
    for item in gallery:
        title = item.get("title","")
        url = item.get("url","#")
        img = item.get("img","")
        desc = item.get("desc","")
        desc_is_html = item.get("desc_is_html", False)
        issues = item.get("issues", [])
        buzz = item.get("buzzword", []) or item.get("buzzwords", [])

        desc_html = desc if desc_is_html else _esc(desc).replace("\n","<br/>")

        buzz_html = ""
        if buzz:
            buzz_html = '<div class="buzzwrap">' + "".join(
                f'<button type="button" class="badge badge-link" data-filter-chip="{_esc(str(b))}">{_esc(str(b))}</button>'
                for b in buzz
            ) + "</div>"

        # === NEW: overlay för issues uppe till höger ===
        issues_overlay = ""
        if issues:
            pills = "".join(
                f'<a class="pill" href="{_esc(i)}" target="_blank" rel="noopener">{_esc(_issue_num(i))}</a>'
                for i in issues
            )
            # GitHub-ikon som går till repo-issues
            gh = f'<a class="pill pill-ghost" href="{_esc(_issues_base(issues[0]))}" target="_blank" rel="noopener" aria-label="GitHub issues">🐙</a>'
            issues_overlay = f'<div class="issues-overlay">{gh}{pills}</div>'

        # Bygg tummen som DIV så vi kan lägga overlay bredvid länken
        thumb_html = f"""
          <div class="thumb">
            <a class="thumb-link" href="{_esc(url)}" target="_blank" rel="noopener">
              <img loading="lazy" src="{_esc(img)}" alt="{_esc(title)}"/>
            </a>
            {issues_overlay}
          </div>
        """

        card = f"""
        <article class="card" data-buzz="{_esc('|'.join([str(b) for b in buzz]))}" data-buzz-norm="{_esc('|'.join([str(b).casefold() for b in buzz]))}">
          {thumb_html}
          <div class="body">
            <h3 class="title"><a href="{_esc(url)}" target="_blank" rel="noopener">{_esc(title)}</a></h3>
            {buzz_html}
            <p class="desc">{desc_html}</p>
          </div>
        </article>
        """
        cards.append(card)

    css = """
    <style>
      .gallery{display:grid;gap:1rem;grid-template-columns:repeat(auto-fill,minmax(320px,1fr));}
      .card{background:#fff;border:1px solid #eee;border-radius:16px;overflow:hidden;box-shadow:0 1px 3px rgba(0,0,0,.06);}
      .thumb{position:relative;display:block;aspect-ratio:16/9;background:#f6f7f8;}
      .thumb-link, .thumb img{display:block;width:100%;height:100%;object-fit:cover;}
      /* === NEW: issues overlay uppe till höger === */
      .issues-overlay{position:absolute;top:.5rem;right:.5rem;display:flex;gap:.5rem;flex-wrap:wrap;justify-content:flex-end;max-width:90%;}
      .pill{display:inline-block;padding:.25rem .6rem;border-radius:999px;background:rgba(255,255,255,.9);border:1px solid rgba(0,0,0,.1);text-decoration:none;font-weight:700}
      .pill-ghost{background:rgba(255,255,255,.85);font-weight:600}
      .pill:hover{background:#fff}
      /* övrigt oförändrat */
      .body{padding:0.9rem 1rem 1rem;}
      .title{margin:.1rem 0 .4rem 0;font-size:1.05rem;line-height:1.2}
      .title a{text-decoration:none;color:#111;}
      .desc{margin:.5rem 0;color:#333}
      .buzzwrap{display:flex;flex-wrap:wrap;gap:.4rem;margin:.25rem 0 .5rem}
      .badge{display:inline-block;font-size:.78rem;padding:.18rem .5rem;border-radius:999px;border:1px solid #ddd;background:#fafafa}
      .badge-link{cursor:pointer}
      .card[hidden]{display:none !important;}
    </style>
    """

    # Inline JS för filtrering
    js = """
    <script>
    (function(){
      const norm = s => (s || "").toLocaleLowerCase().normalize("NFKC");
      const chips = Array.from(document.querySelectorAll('[data-chip]'));
      const clearBtn = document.getElementById('clearFilters');
      const cards = Array.from(document.querySelectorAll('.card'));
      let active = new Set();

      function applyFilter(){
        if(active.size === 0){
          cards.forEach(c => c.hidden = false);
          return;
        }
        const wanted = Array.from(active).map(norm);
        cards.forEach(card=>{
          const has = (card.getAttribute('data-buzz-norm') || '');
          const tags = has.split('|').filter(Boolean);
          // OCH-logik: alla valda måste finnas på kortet
          const ok = wanted.every(w => tags.includes(w));
          card.hidden = !ok;
        });
      }

      function toggleChip(el){
        const val = el.getAttribute('data-chip');
        if(el.classList.contains('active')){
          el.classList.remove('active');
          active.delete(val);
        } else {
          el.classList.add('active');
          active.add(val);
        }
        applyFilter();
      }

      chips.forEach(ch => ch.addEventListener('click', ()=>toggleChip(ch)));

      // Klick på badge inne i kort = aktivera motsvarande chip
      document.addEventListener('click', e=>{
        const b = e.target.closest('[data-filter-chip]');
        if(!b) return;
        const val = b.getAttribute('data-filter-chip');
        const chip = chips.find(c => norm(c.getAttribute('data-chip')) === norm(val));
        if(chip) chip.click();
      });

      if(clearBtn){
        clearBtn.addEventListener('click', ()=>{
          chips.forEach(c => c.classList.remove('active'));
          active.clear();
          applyFilter();
        });
      }
    })();
    </script>
    """

    html_out = (
        css +
        filter_bar +
        '<section class="gallery">' + "\n".join(cards) + "</section>" +
        js
    )
    return html_out


In [ ]:
import html
from urllib.parse import urlparse

def _render_map_gallery_html(gallery):
    def _esc(s):
        return html.escape(str(s), quote=True)

    # --- Hjälpare för issues ---
    def _issue_num(u: str) -> str:
        # Returnerar t.ex. "#132"
        try:
            parts = [p for p in urlparse(u).path.split("/") if p]
            if "issues" in parts:
                return f"#{parts[parts.index('issues')+1]}"
        except Exception:
            pass
        return "#"

    def _issues_base(u: str) -> str:
        # Bas-URL till repo-issues-sidan
        try:
            pre, _ = u.split("/issues/", 1)
            return pre + "/issues"
        except ValueError:
            return u

    # --- Samla alla buzzwords (för filterchips) ---
    all_buzz = []
    for item in gallery:
        buzz = item.get("buzzword", []) or item.get("buzzwords", [])
        all_buzz.extend([str(b).strip() for b in buzz])

    # Unika, sorterade buzzwords (skonsam sortering, behåll original-case)
    seen = set()
    uniq_buzz = []
    for b in all_buzz:
        key = b.casefold()
        if key not in seen:
            seen.add(key)
            uniq_buzz.append(b)
    uniq_buzz.sort(key=lambda x: x.casefold())

    def _buzz_attrs(buzz_list):
        # Data-attribut för JS-filtrering; lagra både original och normaliserad
        raw = "|".join(buzz_list)
        norm = "|".join([x.casefold() for x in buzz_list])
        return f'data-buzz="{_esc(raw)}" data-buzz-norm="{_esc(norm)}"'

    # --- Bygg korten ---
    cards = []
    for item in gallery:
        title = item.get("title","")
        url = item.get("url","#")
        img = item.get("img","")
        desc = item.get("desc","")
        desc_is_html = item.get("desc_is_html", False)
        issues = item.get("issues", [])
        buzz = item.get("buzzword", []) or item.get("buzzwords", [])

        desc_html = desc if desc_is_html else _esc(desc).replace("\n","<br/>")

        buzz_html = ""
        if buzz:
            buzz_html = '<div class="buzzwrap">' + "".join(
                f'<button type="button" class="badge badge-link" data-filter-chip="{_esc(str(b))}">{_esc(str(b))}</button>'
                for b in buzz
            ) + "</div>"

        # --- NEW: overlay för issues uppe till höger ---
        issues_overlay = ""
        if issues:
            pills = "".join(
                f'<a class="pill" href="{_esc(i)}" target="_blank" rel="noopener">{_esc(_issue_num(i))}</a>'
                for i in issues
            )
            gh = f'<a class="pill pill-ghost" href="{_esc(_issues_base(issues[0]))}" target="_blank" rel="noopener" aria-label="GitHub issues">🐙</a>'
            issues_overlay = f'<div class="issues-overlay">{gh}{pills}</div>'

        # Gör tumnageln till en DIV så overlay kan positioneras inne i den
        thumb_html = f"""
          <div class="thumb">
            <a class="thumb-link" href="{_esc(url)}" target="_blank" rel="noopener">
              <img loading="lazy" src="{_esc(img)}" alt="{_esc(title)}"/>
            </a>
            {issues_overlay}
          </div>
        """

        card = f"""
        <article class="card" {_buzz_attrs([str(b) for b in buzz])}>
          {thumb_html}
          <div class="body">
            <h3 class="title"><a href="{_esc(url)}" target="_blank" rel="noopener">{_esc(title)}</a></h3>
            {buzz_html}
            <p class="desc">{desc_html}</p>
          </div>
        </article>
        """
        cards.append(card)

    # --- Bygg filterbaren ---
    filter_bar = ""
    if uniq_buzz:
        chips = "".join(
            f'<button type="button" class="chip" data-chip="{_esc(b)}">{_esc(b)}</button>'
            for b in uniq_buzz
        )
        filter_bar = f"""
        <div class="filters">
          <div class="chips" id="buzzChips">{chips}</div>
          <div class="actions">
            <button type="button" id="clearFilters" class="clear">Rensa filter</button>
          </div>
        </div>
        """

    # --- Stilar (inkl. overlay) ---
    css = """
    <style>
      .filters{display:flex;flex-wrap:wrap;align-items:center;justify-content:space-between;gap:.6rem;margin:.5rem 0 1rem}
      .chips{display:flex;flex-wrap:wrap;gap:.4rem}
      .chip{border:1px solid #ddd;background:#fafafa;border-radius:999px;padding:.28rem .65rem;font-size:.85rem;cursor:pointer}
      .chip.active{background:#111;color:#fff;border-color:#111}
      .clear{border:1px solid #ddd;background:#fff;border-radius:8px;padding:.35rem .6rem;font-size:.85rem;cursor:pointer}
      .gallery{display:grid;gap:1rem;grid-template-columns:repeat(auto-fill,minmax(320px,1fr));}
      .card{background:#fff;border:1px solid #eee;border-radius:16px;overflow:hidden;box-shadow:0 1px 3px rgba(0,0,0,.06);}
      .thumb{position:relative;display:block;aspect-ratio:16/9;background:#f6f7f8;}
      .thumb-link, .thumb img{display:block;width:100%;height:100%;object-fit:cover;}
      /* Issues overlay uppe till höger */
      .issues-overlay{position:absolute;top:.5rem;right:.5rem;display:flex;gap:.5rem;flex-wrap:wrap;justify-content:flex-end;max-width:90%;}
      .pill{display:inline-block;padding:.25rem .6rem;border-radius:999px;background:rgba(255,255,255,.92);border:1px solid rgba(0,0,0,.12);text-decoration:none;font-weight:700}
      .pill-ghost{background:rgba(255,255,255,.85);font-weight:600}
      .pill:hover{background:#fff}
      .body{padding:0.9rem 1rem 1rem;}
      .title{margin:.1rem 0 .4rem 0;font-size:1.05rem;line-height:1.2}
      .title a{text-decoration:none;color:#111;}
      .desc{margin:.5rem 0;color:#333}
      .buzzwrap{display:flex;flex-wrap:wrap;gap:.4rem;margin:.25rem 0 .5rem}
      .badge{display:inline-block;font-size:.78rem;padding:.18rem .5rem;border-radius:999px;border:1px solid #ddd;background:#fafafa}
      .badge-link{cursor:pointer}
      .card[hidden]{display:none !important;}
    </style>
    """

    # --- JS för filtrering (OCH-logik) ---
    js = """
    <script>
    (function(){
      const norm = s => (s || "").toLocaleLowerCase().normalize("NFKC");
      const chips = Array.from(document.querySelectorAll('[data-chip]'));
      const clearBtn = document.getElementById('clearFilters');
      const cards = Array.from(document.querySelectorAll('.card'));
      let active = new Set();

      function applyFilter(){
        if(active.size === 0){
          cards.forEach(c => c.hidden = false);
          return;
        }
        const wanted = Array.from(active).map(norm);
        cards.forEach(card=>{
          const has = (card.getAttribute('data-buzz-norm') || '');
          const tags = has.split('|').filter(Boolean);
          const ok = wanted.every(w => tags.includes(w));
          card.hidden = !ok;
        });
      }

      function toggleChip(el){
        const val = el.getAttribute('data-chip');
        if(el.classList.contains('active')){
          el.classList.remove('active');
          active.delete(val);
        } else {
          el.classList.add('active');
          active.add(val);
        }
        applyFilter();
      }

      chips.forEach(ch => ch.addEventListener('click', ()=>toggleChip(ch)));

      // Klick på badge inne i kort = aktivera motsvarande chip
      document.addEventListener('click', e=>{
        const b = e.target.closest('[data-filter-chip]');
        if(!b) return;
        const val = b.getAttribute('data-filter-chip');
        const chip = chips.find(c => norm(c.getAttribute('data-chip')) === norm(val));
        if(chip) chip.click();
      });

      if(clearBtn){
        clearBtn.addEventListener('click', ()=>{
          chips.forEach(c => c.classList.remove('active'));
          active.clear();
          applyFilter();
        });
      }
    })();
    </script>
    """

    html_out = (
        css +
        filter_bar +
        '<section class="gallery">' + "\n".join(cards) + "</section>" +
        js
    )
    return html_out


In [ ]:
gallery = [
    {
        "title": "🚻 Toaletter nära leden",
        "url": "https://raw.githack.com/salgo60/Stockholm_Archipelago_Trail/main/notebook/output/Issue_132_2_toaletter_nara_stockholm_archipelago_trail_latest.html",
        "img": "https://raw.githubusercontent.com/salgo60/Stockholm_Archipelago_Trail/main/notebook/output/thumbs/SAT_Map_132_toilets.jpg",
        "desc": "🚻 Söker i OpenStreetMap efter toaletter längs leden och ser vilket metadata "
        "som saknas. Idag saknas realtidsinfo och kopplingar till aktuell status – trots att "
        "Skärgårdsstiftelsen fått 1 miljon Euro för digitalisering."
        "<br/><br /><b>Önskvärt vore stöd för:</b>"
        "<ll><li>🧑‍🤝‍🧑🗂️ projektytor och öppna backlogs annars blir det ingen interoperabilitet"
        "<li>🌍 data drivna plattformar och inga datasilos - idag ställer Tillväxtsverket inga krav på koppling hjärtstartare <b>varför</b></li>"
        "<li>🛰️ ETT API för felanmälan</li>"
        "<li>🖼️ koppling till bilddatabaser med fria bilder för hjärtstartare - länkade data</li>"
        "<li>🔥 att Naturvårdsverket visar på digital mognad och har med <b>Toaletter som friluftslivsdata</b> som skall levereras"
        "</ll>",
        "desc_is_html": True,
        "issues": [
            "https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/93",
            "https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/132",
            "https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/140",
            "https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/127",
            "https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/171"
        ],
        "buzzwords": ["APIFirst","Bilddatabas","Ekosystem", "Flerspråkighet","Smart tourism", "Öppen data"]
    },
    {
        "title": "💧 Dricksvatten nära leden",
        "url": "https://raw.githack.com/salgo60/Stockholm_Archipelago_Trail/main/kartor/Issue_139_dricksvatten_nara_stockholm_archipelago_trail_latest.html",
        "img": "https://raw.githubusercontent.com/salgo60/Stockholm_Archipelago_Trail/main/notebook/output/thumbs/SAT_Map_139_drinkingwater.jpg",
        "desc": "💧 Söker i OpenStreetMap efter dricksvatten längs leden. "
        "Idag saknas ett ekosystem där man digitalt kan se vad som är påslaget och testat. "
        "Öppna data från Skärgårdsstiftelsen borde ge exempel på hur skärgården beskrivs digitalt."
        "<br/><br /><b>Önskvärt vore stöd för:</b>"
        "<ll><li>🧑‍🤝‍🧑🗂️ projektytor och öppna backlogs annars blir det ingen interoperabilitet"
        "<li>🌍 data drivna plattformar och inga datasilos - idag ställer Tillväxtsverket inga krav på koppling hjärtstartare <b>varför</b></li>"
        "<li>🛰️ ETT API för felanmälan</li>"
        "<li>🖼️ koppling till bilddatabaser med fria bilder för hjärtstartare - länkade data</li>"
        "<li>🌍 'samma som' dvs. Linked data där vi skall kunna följa från ansökan - bidrag -leverans inte dagens pdf:er</li>"
        "<li>🛰️ landningssidor för alla leverabler och <a href='https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/171' "
        "target='_blanket'>persistenta identifierare</a></li>"
        "<li>🔥 att Naturvårdsverket visar på digital mognad och har med <b>tillgång till dricksvatten som friluftslivsdata</b>"
        "</ll>",
        "desc_is_html": True,
        "issues": [
            "https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/93",
            "https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/139",
            "https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/140",
            "https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/171"
        ],
        "buzzwords": ["APIFirst","Bilddatabas", "Ekosystem", "Flerspråkighet","Interoperabilitet","Öppen data"]
    },
    {
        "title": "🖼️ Koppling mellan led och bild – Image Quality Control",
        "url": "https://raw.githack.com/salgo60/Stockholm_Archipelago_Trail/main/notebook/output/SAT_ALL_IN_ONE_142_3_image_qc_latest.html",
        "img": "https://raw.githubusercontent.com/salgo60/Stockholm_Archipelago_Trail/main/notebook/output/thumbs/SAT_Image_Quality_Control_400x225.jpg",
        "desc": "🖼️ Hela leden är dokumenterad i OSM där varje etapp "
        "och delsträcka kopplas till bilder på Wikimedia Commons. "
        "Denna karta låter dig kvalitetssäkra både kartinfo och bildmaterial. "
        "Innehåller lager för 🚻 toaletter och 💧 dricksvatten."
        "<br/><br /><b>Önskvärt vore stöd för:</b> "
        "<ll><li>🌍 flerspråkigt data</li>"
        "<li>♿ tydligare tillgänglighet</li>"
        "<li>🛰️ API för felanmälan</li>"
        "<li>🖼️ bilddatabaser med fria bilder och länkade data</li></ll>",
        "desc_is_html": True,
        "issues": ["https://github.com/salgo60/ProjectOutdoorGyms/issues/74",
                  "https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/142"],
        "buzzwords": ["APIFirst","Bilddatabas","Ekosystem", "Interoperabilitet", "Öppen data"]
    },
    {
        "title": "♿ Funktionstillgänglighet – rullstol",
        "url": "https://raw.githack.com/salgo60/Stockholm_Archipelago_Trail/main/notebook/output/SAT_WHEELCHAIR_073_wheelchair_latest.html",
        "img": "https://raw.githubusercontent.com/salgo60/Stockholm_Archipelago_Trail/main/notebook/output/thumbs/SAT_Wheelchair_thumbnail.jpg",
        "desc": "♿ Visar objekt längs leden som är taggade 'wheelchair' i OSM. "
        "Även här efterlyses bättre öppna data från Skärgårdsstiftelsen. "
        "<strong>Naturvårdsverket</strong> som alla tror är expertmyndighet ser vi har funderat sedan 2019. "
        "När till och med Myndigheten för delaktighet sitter med i Myndighetsnätverket om vandringsleder men man sedan 2019 inte lyckats "
        "ställa tydliga krav på tillgänglighet eller leverera data som beskriver detta – då är det uppenbart att det är fel laguppställning. "
        "Fortsätter det så här kommer ingenting att vara på plats ens 2035. Hur svårt kan det vara... med chatGPT 5 sekunder.."
        "<br/><br /><b>Önskvärt vore stöd för:</b> "
        "<ll><li>🧑‍🤝‍🧑🗂️ projektytor och öppna backlogs annars blir det ingen interoperabilitet<li>🌍 flerspråkigt data</li>"
        "<li>♿ tydligare tillgänglighet</li>"
        "<li>🛰️ API för felanmälan</li>"
        "<li>🖼️ koppling till bilddatabaser med fria bilder</li></ll>",
        "desc_is_html": True,
        "issues": [
            "https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/73",
            "https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/81",
            "https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/93",
            "https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/171",
            "https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/172",
            "https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/174"
        ],
        "buzzwords": ["Flerspråkighet","Ekosystem", "Interoperabilitet","Tillgänglighet", "Öppen data"]
    },
    {
        "title": "✅ Att göra i OSM – Todo",
        "url": "https://raw.githack.com/salgo60/Stockholm_Archipelago_Trail/main/notebook/output/SAT_ALL_IN_ONE_142_3_ALL_latest.html",
        "img": "https://raw.githubusercontent.com/salgo60/Stockholm_Archipelago_Trail/main/notebook/output/thumbs/SAT_Audit_layers_thumbnail.jpg",
        "desc": "✅ Visualiserar saknade OSM-taggar på en karta över leden.",
        "issues": ["https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/142"],
        "buzzwords": ["Interoperabilitet", "Öppna API:er"]
    },
    {
        "title": "🔍 Audit-lager i OSM",
        "url": "https://raw.githack.com/salgo60/Stockholm_Archipelago_Trail/main/notebook/output/SAT_ALL_IN_ONE_142_3_split_todo_latest.html",
        "img": "https://raw.githubusercontent.com/salgo60/Stockholm_Archipelago_Trail/main/notebook/output/thumbs/SAT_Audit_layers_thumbnail.jpg",
        "desc": "🔍 Karta som visar var OSM-taggar saknas för surface / foot / sac / trail_visibility / step_count.",
        "issues": ["https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/142"],
        "buzzwords": ["Interoperabilitet","Öppna API:er", "Öppen data"]
    },
    {
        "title": "🪜 Trappsteg på leden – Steps",
        "url": "https://raw.githack.com/salgo60/Stockholm_Archipelago_Trail/main/notebook/output/SAT_ALL_IN_ONE_142_3_steps_only_latest.html",
        "img": "https://raw.githubusercontent.com/salgo60/Stockholm_Archipelago_Trail/main/notebook/output/thumbs/SAT_Steps_Only_thumbnail.jpg",
        "desc": "🪜 SAT-leden har begärt ersättning för ~200 m trappor "
        "– men de är svåra att hitta. Är det detta som är överraskningen att inget levereras?",
        "issues": [
            "https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/148",
            "https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/171"
        ],
        "buzzwords": ["Bilddatabas", "Flerspråkighet","Interoperabilitet","Tillgänglighet"]
    },
    {
        "title": "🌊 Närhet till vatten – Proximity",
        "url": "https://raw.githack.com/salgo60/Stockholm_Archipelago_Trail/main/notebook/output/SAT_ALL_IN_ONE_142_3_water_proximity_latest.html",
        "img": "https://raw.githubusercontent.com/salgo60/Stockholm_Archipelago_Trail/main/notebook/output/thumbs/SAT_Water_Proximity_800x450.jpg",
        "desc": "🌊 Vissa sträckor går långt från vattnet – på gott och ont. Kartan visar hur nära/långt från vattnet olika delar ligger, samt längsta distans från vatten.",
        "desc_is_html": True,
        "issues": [
            "https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/142",
            "https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/171"
        ],
        "buzzwords": [ "Interoperabilitet", "Smart tourism","Öppen data"]
    },
    {
        "title": "📚 Wikipedia/Wikidata",
        "url": "https://raw.githack.com/salgo60/Stockholm_Archipelago_Trail/main/notebook/output/SAT_ALL_IN_ONE_142_3_Wikipedia.html",
        "img": "https://raw.githubusercontent.com/salgo60/Stockholm_Archipelago_Trail/main/notebook/output/thumbs/SAT_Map_Wikipedia_400x225.jpg",
        "desc": "📚 Mer än 600 Wikipedia/Wikidata-objekt har kopplats till SAT-leden. "
        "Tanken är att visa på hur <strong>digital interoperabilitet</strong> fungerar i praktiken, "
        "och hur man 2025 enkelt kan leverera flerspråkighet med vettiga ekosystem som hanterar"
        "persistenta identifierare och api:er <br/><br /><b>Önskvärt vore stöd för:</b> "
        "<ll><li>🌍 flerspråkig data</li>"
        "<li>🖼️ koppling till bilddatabaser med fria bilder</li>"
        "<li>🛰️ Länkade data samma som jmf <a href='https://github.com/salgo60/NOSAD-POC-Wikidata/issues/13' target=_blank >hur vi jobbar med Nobelprize.org</a> och uppdaterar Wikipedia via Wikidata</li>"
        "</ll>",
        "desc_is_html": True,
        "issues": [
            "https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/22",
            "https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/171"
        ],
        "buzzwords": [ "APIFirst", "Community-driven", "Flerspråkighet", "Interoperabilitet", "Smart tourism","Öppen data", "Öppna API:er"]
    },
    {
        "title": "🪧 SAT infoskyltar",
        "url": "https://raw.githack.com/salgo60/Stockholm_Archipelago_Trail/main/notebook/output/182_WD_OSM_signs_latest.html",
        "img": "https://raw.githubusercontent.com/salgo60/Stockholm_Archipelago_Trail/main/notebook/output/thumbs/SAT_InfoSign_400x225.jpg",
        "desc": "🪧 135 turistinformationsskyltar beställdes via Tillväxtverket – endast ~80 hittade. "
        "Ingen öppen data om placering eller leveransstatus. "
        "Vi skall överraskas - ja vi är överraskade hur dumt skattepengar slösas "
        "<i>The Magic slöseri</i><br/>"
        "<br/><br /><b>Önskvärt vore stöd för:</b> "
        "<ll><li>🧑‍🤝‍🧑🗂️ projektytor och öppna backlogs annars blir det ingen interoperabilitet"
        "<li>🌍 flerspråkig data</li>"
        "<li>♿ tydligare tillgänglighet</li>"
        "<li>🛰️ API för felanmälan</li>"
        "<li>🖼️ koppling till bilddatabaser med fria bilder</li>"
        "<li>⏱️ information om tid att vandra sträckan</li>"
        "<li>⚠️ status på leden med senaste problem"
        "<li>📱 länk till sida som känner av språkinställning i mobilen och visar info på rätt språk"
        "<li>🍴🏨🛒🚰🚻 länk till sida som visar <b>öppna</b>restauranger, boenden, affärer, dricksvatten, toaletter"
        "</ll>"        ,
        "desc_is_html": True,
        "issues": [
            "https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/81",
            "https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/93",
            "https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/97",
            "https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/176",
            "https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/180",
            "https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/171"
        ],
        "buzzwords": ["Bilddatabas", "Flerspråkighet","Interoperabilitet", "QR-koder", "Smart tourism","Öppen data"]
    },
    {
        "title": "🐾 SAT – iNaturalist",
        "url": "https://raw.githack.com/salgo60/Stockholm_Archipelago_Trail/main/notebook/output/179_inat_taxa_layers_colored_5.html",
        "img": "https://raw.githubusercontent.com/salgo60/Stockholm_Archipelago_Trail/main/notebook/output/thumbs/SAT_179_2_iNaturalist_400x225.jpg",
        "desc": "🐾 Datadriven test: på några minuter hämtas communitydriven artdata från iNaturalist och visar vad som faktiskt finns längs leden. Kontrasten mot långsamma processer är tydlig.",
        "issues": [
            "https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/179",
            "https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/146",
            "https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/148"
        ],
        "buzzwords": ["APIFirst", "Community-driven", "Ekosystem",  "Flerspråkighet" ,"Realtidsdata", "Smart tourism", "Öppna API:er", "Öppen data"]
    },
    {
        "title": "🔥🏕️ SAT grillplatser & vindskydd",
        "url": "https://raw.githack.com/salgo60/Stockholm_Archipelago_Trail/main/notebook/output/map185_vindskydd_grillplatser.html",
        "img": "https://raw.githubusercontent.com/salgo60/Stockholm_Archipelago_Trail/main/notebook/output/thumbs/SAT188_WD_OSM_bbq_400x225.jpg",
        "desc": "🔥🏕️ 8 grillplatser och 11 vindskydd skulle beställas "
        "– men var är de och när levereras de? Kartan visar de som hittats längs leden.<br />"
        "<br/><br /><b>Önskvärt vore stöd för:</b> "
        "<ll><li>🧑‍🤝‍🧑🗂️ projektytor och öppna backlogs annars blir det ingen interoperabilitet"
        "<li>🌍 flerspråkigt data</li>"
        "<li>♿ tydligare tillgänglighet</li>"
        "<li>🛰️ API för felanmälan</li>"
        "<li>🖼️ koppling till bilddatabaser med fria bilder</li></ll>",
        "desc_is_html": True,        
        "issues": [
            "https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/11",
            "https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/49",
            "https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/171",
            "https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/146",
            "https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/148",
            "https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/185"
        ],
        "buzzwords": ["Bilddatabas","Flerspråkighet","Interoperabilitet", "Smart tourism", "Öppen data"]
    },
    {
        "title": "🧭 SAT-leden och Nordsydlinjen",
        "url": "https://umap.openstreetmap.fr/en/map/boende-nordsydlinjen-och-stockholm-archipelago-tra_1257362?scaleControl=true&miniMap=true&scrollWheelZoom=true&zoomControl=true&editMode=disabled&moreControl=true&searchControl=true&tilelayersControl=true&embedControl=null&datalayersControl=true&onLoadPanel=caption&captionBar=true&captionMenus=true&datalayers=35da97a6-893b-46de-b7b3-4ed341b042c3%2Cb3a4a332-36f7-4949-80dd-2468d4a712ea%2C752d2480-e5a5-4fa2-9b78-15c75e545f64%2C7b203db0-53d4-4a1a-9f7d-a298344c6da5&locateControl=true&measureControl=true#8/59.305/18.515",
        "img": "https://raw.githubusercontent.com/salgo60/Stockholm_Archipelago_Trail/main/notebook/output/thumbs/SAT_NordSyd_400x225.jpg",
        "desc": "🧭 Flum som <i>Magic season</i> möter verklighet<br/> SAT-projektet hävdar att NordSyd-linjen ska göra det enkelt att vandra Stockholm Archipelago Trail. <strong>Sanningen</strong>? Nja. Få leder är så splittrade och otydliga som SAT. På kartan ser du hur NordSydlinjen faktiskt går – och var SAT-leden ligger. Och för att krångla till ekvationen ännu mer: SAT vill att vi ska vandra off-season, när NordSydlinjen inte ens går… <br/><b>Resultat:</b> dyra investeringar + flummiga löften = en led som aldrig riktigt hänger ihop. <strong>Tillväxtverket – vakna!</strong> Det handlar om våra skattepengar, som inte skapar verklig nytta utan göder ett osunt bidragstiggeri. Man pratar om “Magic season” och att vi ska “överraskas” – men det vi i praktiken överraskas av är hur oproffsigt skattepengar delas ut till lycksökare som levererar flum istället för resultat.",
        "desc_is_html": True,
        "issues": [
            "https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/81",
            "https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/164",
            "https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/151",
            "https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/149"
        ],
        "buzzwords": ["APIFirst", "Ekosystem", "Öppen data", "Flerspråkighet", "Smart tourism"]
    },    
    {
        "title": "🏕️🌿 SAT – Tältplatser & naturreservat",
        "url": "https://umap.openstreetmap.fr/en/map/stockholm-archipelago-trail-naturreservat-talta_1280785?scaleControl=false&miniMap=false&scrollWheelZoom=false&zoomControl=true&editMode=disabled&moreControl=true&searchControl=null&tilelayersControl=null&embedControl=null&datalayersControl=true&onLoadPanel=none&captionBar=false&captionMenus=true#10/59.0261/18.7015",
        "img": "https://raw.githubusercontent.com/salgo60/Stockholm_Archipelago_Trail/main/notebook/output/thumbs/SAT_Naturreserve_400x225.jpg",
        "desc": "🏕️🌿 Skall man tälta på öarna gäller allemansrätten – "
        "om man inte är i ett naturreservat. 📜 Denna karta visar "
        "Stockholm Archipelago Trail, Naturreservat och länkar till Naturvårdsverkets Skyddad natur och Länsstyrelsens föreskrifter"
        ". Allt detta är på svenska – skall vi <strong>tro att SAT-projektet och Visit Sweden kommer</strong> att skapa en flerspråkig version av dessa sidor? 😉"
        "<br/><br /><b>Önskvärt vore stöd för:</b> "
        "<ll><li>🧑‍🤝‍🧑🗂️ projektytor och öppna backlogs annars blir det ingen interoperabilitet"
        "<li>🌍 flerspråkigt data</li>"
        "<li>♿ tydligare tillgänglighet</li>"
        "<li>🛰️ API för felanmälan</li>"
        "<li>🖼️ koppling till bilddatabaser med fria bilder</li></ll>"
        "<br /><br /><b>Interoperabiltet</b> OSM <-> Wikdata <-> Wikicommons",
        "desc_is_html": True,
        "issues": [
            "https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/81",
            "https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/164",
            "https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/151",
            "https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/149"
        ],
        "buzzwords": ["Bilddatabas", "Community-driven","Smart tourism", "Öppen data", "Flerspråkighet"]
    },
    {
        "title": "🏕️🌿 SAT – Grillplatser.nu ",
        "url": "https://grillplatser.nu/Karta/Lan/Stockholms-lan",
        "img": "https://raw.githubusercontent.com/salgo60/Stockholm_Archipelago_Trail/main/notebook/output/thumbs/SAT_grillplatsernu_400x225.jpg",
        "desc": "🏕️🌿 Öppna data har efterfrågats från <a href='https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/11' target=_blank>SAT 2025-03-19</a> utan svar se hur > 7000 grillplatser "
        "finns samlade av en community med fria bilder. <br/><br /><b>Önskvärt vore stöd för:</b> "
        "<ll><li>🌍 flerspråkig data</li>"
        "<li>♿ tydligare tillgänglighet</li>"
        "<li>🛰️ API för felanmälan</li>"
        "<li>🖼️ koppling till bilddatabaser med fria bilder</li>"
        "<li>🌍 samma som grillplatser.nu OSM Wikidata</li>"
        "<li>🖼️ landningssidor för alla leverabler och <a href='https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/171' "
        "target='_blanket'>persistenta identifierare</a></li>"
        "</ll>"
        "<br /><br /><b>Interoperabiltet</b><br/> OSM <-> Wikdata <-> Grillplatser.nu <-> Wikicommons"
        "<ll><li>OSM <a href='https://wiki.openstreetmap.org/wiki/Sv:Key:ref:grillplatser.nu' "
        "target='_blanket'>Key:ref:grillplatser.nu</a></li>"
        "<li>OSM overpass <a href='https://overpass-turbo.eu/s/2cmU' "
        "target='_blanket'>Key:ref:grillplatser.nu</a> </li>"
        "</ll><br />",
        "desc_is_html": True,
        "issues": [
            "https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/11",
            "https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/171",
            "https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/185",
            "https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/188",

        ],
        "buzzwords": ["Bilddatabas", "Community-driven", "Flerspråkighet", "Interoperabilitet",  "Smart tourism", "Öppen data"]
    },
    {
        "title": "🔥 SAT – Wikidata - Wikipedia hur man jobbar datadrivet",
        "url": "https://wikishootme.toolforge.org/#lat=59.0617461708114&lng=18.431606590747837&zoom=12",
        "img": "https://raw.githubusercontent.com/salgo60/Stockholm_Archipelago_Trail/main/notebook/output/thumbs/SAT_Wikishootme_400x225.jpg",
        "desc": "🔥 Öppna data från SAT lovas till <a target_blank href='https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/49'>Nacka kommun</a> "
        "trovärdigheten är låg när allt är hemligt... ingen hittar vindskydd som utlovats, trappor som byggts..."
        "<br><br><a target=_blank href='https://www.youtube.com/watch?v=A3GnO4kAIos&list=PLNWUKRLAYDeSSHsFOAOy8L_cAtbsOQ41R&index=1'>Video</a> "
        "om hur wikidata i kartan jobbar med bilder, bildbibliotek, > 200 språk versioner. "
        "<br /><br>Idag jobbar Google med <a href='https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/171' target=_blank>Persistenta Identifierare</a> och realtidsinformation med färjetider skall man "
        "prata om <b>Smart turism</b> fungerar det inte bara med filmer på Instagram om <i>Magic season</i>. "
        "<br/><br /><b>Önskvärt vore stöd för:</b> "
        "<ll><li>🧑‍🤝‍🧑🗂️ projektytor och öppna backlogs annars blir det ingen interoperabilitet"
        "<li>🌍 data drivna plattformar och inga datasilos</li>"
        "<li>🌍 'samma som' dvs. Linked data </li>"
        "<li>🔥 att Naturvårdsverket visar på digital mognad och  <a href='https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/171' "
        "target='_blanket'>skapar för vandringsleder det vi ser i Norge/Finland</a></li>"
        "</ll>",
        "desc_is_html": True,
        "issues": [
            "https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/49",
            "https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/143",
            "https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/145",
            "https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/11",
            "https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/161",
            "https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/171"
        
        ],
        "buzzwords": ["APIFirst","Interoperabilitet", "Smart tourism",]
    },
        {
        "title": "🔥 SAT – Ferrystops - datadrivet",
        "url": "https://raw.githack.com/salgo60/Stockholm_Archipelago_Trail/main/notebook/output/187_WD_OSM_ferry_stops_latest.html",
        "img": "https://raw.githubusercontent.com/salgo60/Stockholm_Archipelago_Trail/main/notebook/output/thumbs/SAT_ferry_400x225.jpg",
        "desc": "🔥 Öppna data från SAT dyker inte upp eftersom projektet inte arbetar datadrivet dom tror <i>det är ett vinterjobb</i>. " 
        "Var SAT leden startar och slutar är otydligt utan den knyter ihop bryggor och stigar och är inte en rundslinga. "
        "Den borde vara ett <a target_blank href='https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/161'>Nodnätverk</a>. "
        "<br />Vi ser hela kedjan är pdf och textdokument från Tillväxtverket till utbetalning av skattepengar... och "
        "slutar med filmer på Instagram med <i>Magic season</i> för att sedan sökas nya bidrag..."
        "<br><b>Google</b> förstår detta med datadrivet och <b>Smart turism</b>. Dom jobbar <b>datadrivet</b> med Google Map och har realtidsinformation om alla "
        " Waxholmsbolagets bryggor och har även alla restauranger med recensioner... "
        "<br />Kartan ovan visar färjestopp som är startpunkter för SAT leden hämtat från <a target_blank href='https://www.youtube.com/watch?v=m_9_23jXPoE'>Wikidata</a>."
        "<br/><br /><b>Önskvärt vore stöd för:</b> "
        "<ll><li>🧑‍🤝‍🧑🗂️ projektytor och öppna backlogs annars blir det ingen interoperabilitet"
        "<li>🌍 data drivna plattformar och inga datasilos - idag ger Tillväxtsverket pengar till datasilos <b>varför</b></li>"
        "<li>🛰️ ETT API för felanmälan</li>"
        "<li>🖼️ koppling till bilddatabaser med fria bilder - länkade data</li>"
        "<li>🌍 'samma som' dvs. Linked data </li>"
        "<li>🛰️ landningssidor för alla leverabler och <a href='https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/171' "
        "target='_blanket'>persistenta identifierare</a></li>"
        "<li>🔥 att Naturvårdsverket visar på digital mognad och  <a href='https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/171' "
        "target='_blanket'>skapar för vandringsleder det vi ser i Norge/Finland</a></li>"
        "</ll>",
        "desc_is_html": True,
        "issues": [
            "https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/39",
            "https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/81",
            "https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/158",
            "https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/186",
            "https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/187"
        ],
        "buzzwords": ["APIFirst","Interoperabilitet", "Realtidsdata", "Smart tourism"]
    },
    
        {
        "title": "🔥 SAT – AED - Hjärtstartare <-> Hjärtstartarregistret",
        "url": "https://raw.githack.com/salgo60/Stockholm_Archipelago_Trail/main/notebook/output/191_WD_OSM_AED_latest.html",
        "img": "https://raw.githubusercontent.com/salgo60/Stockholm_Archipelago_Trail/main/notebook/output/thumbs/SAT_AED_191_400x225.jpg",
        "desc": "Hjärtstartare är till viss del uppstyrt av <a href='https://www.hjartstartarregistret.se/#/' target=_blank>Hjärtstartarregistret</a>  men lång ifrån alla apparater finns där. "
        "Jag har inventerat en del genom dels kolla i registret men även pratat med månniskor längs leden och i OSM."
        "<br><ll><li>Bilder på <a href='https://commons.wikimedia.org/wiki/Category:Stockholm_Archipelago_Trail_AED' target=_blank>Hjärtstartare längs leden</a> "
         "/ <a href='https://wikimap.toolforge.org/?cat=Stockholm_Archipelago_Trail_AED&subcats=true&subcatdepth=3&cluster=false' target=_blank> på en karta</a>"
        "<li>Jag har översatt en polsk app till svenska <a href='https://openaedmap.org/sv/#map=8.89/59.2745/18.942' target=_blank>OpenAED</a> som hämtar sitt data från OSM om AED:er "
        "se issue <a target=_blank href='https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/46#issuecomment-2820021924'>#46</a>" 
        "<li>Haft diskussion 2025-juni-19 13:30 med <a href='https://www.hjartstartarregistret.se/#/' target=_blank>hjärtstartarregistret</a> om bättre integration med OSM <a href='https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/79#issuecomment-2955570965'>#79</a>... "
        "status vi väntar på dom - tveksamt om dom har resurser"
        "<li>Skapat poster i OSM och Wikidata för AED hittade se <a href='https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/191'>#191</a>"
        "</lL>"
        "<br><br><b>Utmaningar jag ser</b>"
        "<ll><li>Något som borde vara självklart att ha ordning på drivs av en organisation <a href='https://www.hlr.nu/'>HLR rådet</a>"
        "<li>var i <a href='https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/46#issuecomment-2833502212' target=_blank>april i Nice</a> där AED:er finns som staten ansvarar för"
        " känns mycket bättre och <a href='https://openaedmap.org/sv/#map=12.71/43.68708/7.2744&node_id=12902177339' target=_blank>AED:er fanns på strandpromenaden</a>"    
        "<li>Hjärtstartarregistret använder idag Open Street Map kartor och skulle vinna på "
        "bättre integration men är nog inte datadrivna och har små resurser"
        "<li><b>SAT projektet har inte ens en projektyta</b> så vad dom vill berättas ostrukturerat på instagram meddelanden... - galet"
        "<li><b>Naturvårdsverket har projektet sedan 2019 om vandringsleder</b> som definierat när man skall mötas "
        "inte hur data om friluftsliv skall beskrivas... - galet se "
        "<a href='https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/144' target=_blank>ChatGPT om nationell samverkansmodell vandringsutveckling</a>"
        "</lL>"
        "<br />Saker som detta måste styras upp men jag ser ingen som kan detta... "
        "ser inte heller att Tillväxtverket bidrar med krav till Naturvårdsverket om bättre info"
        "<br/><br /><b>Önskvärt vore stöd för:</b>"
        "<ll><li>🧑‍🤝‍🧑🗂️ projektytor och öppna backlogs annars blir det ingen interoperabilitet"
        "<li>🌍 data drivna plattformar och inga datasilos - idag ställer Tillväxtsverket inga krav på koppling hjärtstartare <b>varför</b></li>"
        "<li>🛰️ ETT API för felanmälan</li>"
        "<li>🖼️ koppling till bilddatabaser med fria bilder för hjärtstartare - länkade data</li>"
        "<li>🌍 'samma som' dvs. Linked data där vi skall kunna följa från ansökan - bidrag -leverans inte dagens pdf:er</li>"
        "<li>🛰️ landningssidor för alla leverabler och <a href='https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/171' "
        "target='_blanket'>persistenta identifierare</a></li>"
        "<li>🔥 att Naturvårdsverket visar på digital mognad och har med <b>hjärtstartare för friluftslivsdata</b>"
        "</ll>",
        "desc_is_html": True,
        "issues": [
            "https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/191",
            "https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/46",
            "https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/144"
        ],
        "buzzwords": ["APIFirst","Interoperabilitet", "Ekosystem","Realtidsdata", "Smart tourism"]
    }
]

In [ ]:
intro_html = _render_intro_html("""
<ul style="list-style-type: none; padding-left: 0; font-family: Arial, sans-serif; font-size:14px; line-height:1.6;">
  <li style="margin-bottom:6px;">🔗 Projektyta <a href="https://github.com/salgo60/Stockholm_Archipelago_Trail/issues?q=is%3Aissue" target="_blank">GITHUB</a></li>
  <li style="margin-bottom:6px;">🖼️ <a href="https://commons.wikimedia.org/wiki/Category:Stockholm%20Archipelago%20Trail" target="_blank">Bilder på leden</a> – Wikicommons - <a href="https://wikimap.toolforge.org/?cat=Stockholm_Archipelago_Trail&subcats=true&subcatdepth=4&cluster=false">Karta (tar tid)</a></li>
  <li style="margin-bottom:6px;">👣 Icke officiell <a href="https://www.facebook.com/groups/2875020699552247" target="_blank">FB grupp om leden av vandrare för vandrare</a></li>
  <li style="margin-bottom:6px;">📚 <a href="https://www.wikidata.org/wiki/Wikidata:WikiProject_Stockholm_Archipelago_Trail" target="_blank">Wikipedia projekt</a> / Umap: <a target=_blank href="https://umap.openstreetmap.fr/en/map/boende-nordsydlinjen-och-stockholm-archipelago-tra_1257362?scaleControl=true&miniMap=true&scrollWheelZoom=true&zoomControl=true&editMode=disabled&moreControl=true&searchControl=true&tilelayersControl=true&embedControl=null&datalayersControl=true&onLoadPanel=caption&captionBar=true&captionMenus=true&datalayers=35da97a6-893b-46de-b7b3-4ed341b042c3%2Cb3a4a332-36f7-4949-80dd-2468d4a712ea%2C752d2480-e5a5-4fa2-9b78-15c75e545f64%2C7b203db0-53d4-4a1a-9f7d-a298344c6da5&locateControl=true&measureControl=true">NordSyd</a>, <a target=_blank href="https://umap.openstreetmap.fr/en/map/stockholm-archipelago-trail-naturreservat-talta_1280785?scaleControl=false&miniMap=false&scrollWheelZoom=false&zoomControl=true&editMode=disabled&moreControl=true&searchControl=null&tilelayersControl=null&embedControl=null&datalayersControl=true&onLoadPanel=none&captionBar=false&captionMenus=true">Tälta</a></li>
  <li style="margin-bottom:6px;">🔗 Video om denna yta <a href="https://youtu.be/vy_746Wn4pc" target="_blank">youtube</a> / <a target="_blank" href="https://www.youtube.com/playlist?list=PLNWUKRLAYDeSSHsFOAOy8L_cAtbsOQ41R">SAT spelista</a></li>
  <li style="margin-bottom:6px;">🔗 Samtal med: <a href="https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/146" target="_blank">Tillväxtverket</a>, <a href="https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/174" target="_blank">Naturvårdsverket</a> om dom brister vi ser.
</ul>
""")




dash_html = make_sat_dashboard(
        etapp_gdfs,
        OUTPUT_DIR=OUTPUT_DIR,
        PROJECT_NAME=PROJECT_NAME,
        stamp=stamp,
        intro_html=intro_html,
        map_gallery=gallery
    )
print("✅ Dashboard with gallery =>", dash_html)

In [ ]:
 # End timer and calculate duration
end_time = time.time()
elapsed_time = end_time - start_time# Bygg audit-lager för den här etappen

# Print current date and total time
print("Date:", datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
minutes, seconds = divmod(elapsed_time, 60)
print("Total time elapsed: {:02.0f} minutes {:05.2f} seconds".format(minutes, seconds))
